In [220]:
#setup Odoo

import requests

jsonrpcid = 0
url = "https://demo-28-08-6.adhoc.ar/jsonrpc"
db = "demo-28-08-6"
username = "Chargebee"
password = "Chargebee"
headers = {'Content-Type': 'application/json'}


class Odoo():

    def __init__(self):
        self.jsonrpcid = 0

    def call(self, method: str, service: str, *args):
        self.jsonrpcid += 1

        data = {
            "jsonrpc": "2.0",
            "method": "call",
            "params": {
                "service": service,
                "method": method,
                "args": args
            },
            "id": self.jsonrpcid
        }

        # Realizar solicitud HTTP POST
        response = requests.post(url, headers=headers, json=data)
        result = response.json()

        return result

    def login(self, db: str, username: str, password: str):
        self.db = db
        self.username = username
        self.password = password
        outcome = self.call('login', 'common', self.db, self.username, self.password)
        if 'result' not in outcome:
            raise Exception('Failed to login')
        self.uid = outcome['result']

    def execute(self, *args):
        result = self.call('execute_kw', 'object', self.db, self.uid, self.password, *args)
        r = result.get('result', False)
        if not r:
            print(result)
            raise Exception('Command has failed')
        return r

    def read(self, model: str, ids: list=[], fields: list=[], context: dict={}):
        return self.execute(model, 'read', [ids, fields], {'context': context})

    def search_read(self, model: str, domain: list=[], fields: list=[], offset: int=0, limit: int=10, context: dict={}):
        return self.execute(model, 'search_read', [domain, fields, offset, limit], {'context': context})

    def create(self, model: str, vals_list: list=[], context: dict={}):
        return self.execute(model, 'create', vals_list, {'context': context})

# Crear nueva instancia
odoo = Odoo()

# Realizar login
odoo.login(db, username, password)

In [2]:
#setup chargebee

import chargebee

chargebee.configure('live_3NEiAmTg7dQArgRHR04LvHaIABupcuQOS', 'cliengo')

In [3]:
# Obtener las invoices desde Chargebee

import pandas as pd
import time
def get_chargebee_invoices_august_2024():
    invoice_list = []
    offset = None  # Inicialmente sin offset para la paginación
    start_date = int(time.mktime(time.strptime("2024-08-01", "%Y-%m-%d")))  # 1 de agosto de 2024 en Unix Timestamp
    end_date = int(time.mktime(time.strptime("2024-08-31 23:59:59", "%Y-%m-%d %H:%M:%S")))  # 31 de agosto de 2024

    while True:
        # Obtener la lista de facturas, sin límite explícito, pero con paginación
        entries = chargebee.Invoice.list({
            "status[in]": ["paid", "payment_due"],
            "date[between]": [start_date, end_date],  # Filtro por rango de fechas en Unix Timestamp
            "sort_by[asc]": "date",
            "limit": 100,  # Procesar 100 facturas por página
            "offset": offset  # Usar el offset actual para paginación
        })

        # Agregar las facturas a la lista
        for entry in entries:
            invoice = entry.invoice
            invoice_list.append(invoice.__dict__)  # Agregar la factura a la lista

        # Verificar si hay más páginas (si existe un offset para la siguiente página)
        offset = entries.next_offset
        if not offset:
            break  # Salir del bucle si no hay más páginas

    return invoice_list

# Llamar la función para obtener todas las facturas de agosto de 2024
invoices = get_chargebee_invoices_august_2024()

# Convertir la lista de diccionarios en un DataFrame de pandas
df = pd.DataFrame(invoices)

# Mostrar el DataFrame
df

,values,sub_types,dependant_types,id,po_number,customer_id,subscription_id,recurring,status,vat_number,...,void_reason_code,deleted,tax_category,vat_number_prefix,channel,business_entity_id,site_details_at_creation,tax_origin,object,base_currency_code
0,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
1,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2,"{'id': '185394', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,True,paid,30025707,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
3,"{'id': '185395', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185395,Argentina - Transferencia Bancaria,5cb4d5d0e4b0bd6bed7869fa,16CY2RUEdBvEE1YCI,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
4,"{'id': '185396', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},185396,Chile,5b47de04e4b03c859495a0f7,Izy9TIBR04obyPV06,True,paid,969726408,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,"{'id': '187908', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187908,Argentina - Transferencia Bancaria (Leadaki),5f3da9f3b68252002aaac99e,AzZsu3UBeV4iY8HEC,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2473,"{'id': '187910', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187910,Argentina - Transferencia Bancaria (Leadaki),5a71ba83e4b07b95e31fc272,AzqBxKUBeW38u8EGG,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2474,"{'id': '187892', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187892,Argentina - Transferencia Bancaria (Leadaki),57bddbafe4b08f5c877fe039,169v0eUBcr4Bg73Qq,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2475,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD


In [5]:
#para separar los campos dentro de jsons y diccionarios genere el siguiente codigo @Yepes
def print_column_types(dataframe):
    for column in dataframe.columns:
        dtype = dataframe[column].dtype
        print(f"{column} \t {dtype}")

# Llamar a la función para mostrar las columnas y tipos de datos
print_column_types(df)

values 	 object
sub_types 	 object
dependant_types 	 object
id 	 object
po_number 	 object
customer_id 	 object
subscription_id 	 object
recurring 	 bool
status 	 object
vat_number 	 object
price_type 	 object
date 	 int64
due_date 	 int64
net_term_days 	 int64
exchange_rate 	 float64
currency_code 	 object
total 	 int64
amount_paid 	 int64
amount_adjusted 	 int64
write_off_amount 	 int64
credits_applied 	 int64
amount_due 	 int64
paid_at 	 float64
dunning_status 	 object
next_retry_at 	 float64
voided_at 	 object
resource_version 	 int64
updated_at 	 int64
sub_total 	 int64
sub_total_in_local_currency 	 object
total_in_local_currency 	 object
local_currency_code 	 object
tax 	 int64
local_currency_exchange_rate 	 object
first_invoice 	 bool
new_sales_amount 	 float64
has_advance_charges 	 bool
term_finalized 	 bool
is_gifted 	 bool
generated_at 	 int64
expected_payment_date 	 float64
amount_to_collect 	 int64
round_off_amount 	 int64
line_items 	 object
discounts 	 object
line_item_di

In [6]:
#muestra todos los campos de tipo object ya que entre ellos estan esos campos a desanidar
df_objects = df.select_dtypes(include=['object'])
df_objects

,values,sub_types,dependant_types,id,po_number,customer_id,subscription_id,status,vat_number,price_type,...,payment_owner,void_reason_code,tax_category,vat_number_prefix,channel,business_entity_id,site_details_at_creation,tax_origin,object,base_currency_code
0,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,paid,None,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
1,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,paid,None,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2,"{'id': '185394', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,paid,30025707,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
3,"{'id': '185395', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185395,Argentina - Transferencia Bancaria,5cb4d5d0e4b0bd6bed7869fa,16CY2RUEdBvEE1YCI,paid,None,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
4,"{'id': '185396', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},185396,Chile,5b47de04e4b03c859495a0f7,Izy9TIBR04obyPV06,paid,969726408,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,"{'id': '187908', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187908,Argentina - Transferencia Bancaria (Leadaki),5f3da9f3b68252002aaac99e,AzZsu3UBeV4iY8HEC,paid,None,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2473,"{'id': '187910', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187910,Argentina - Transferencia Bancaria (Leadaki),5a71ba83e4b07b95e31fc272,AzqBxKUBeW38u8EGG,paid,None,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2474,"{'id': '187892', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187892,Argentina - Transferencia Bancaria (Leadaki),57bddbafe4b08f5c877fe039,169v0eUBcr4Bg73Qq,paid,None,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2475,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,paid,None,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD


In [7]:
#aqui separe especificamente estos 2 campos debido a que unos son json y otros son diccionarios, en este caso son 
dict_df = df_objects[['values', 'sub_types']]

In [8]:
dict_df

,values,sub_types
0,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...
1,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...
2,"{'id': '185394', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...
3,"{'id': '185395', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...
4,"{'id': '185396', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...
...,...,...
2472,"{'id': '187908', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...
2473,"{'id': '187910', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...
2474,"{'id': '187892', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...
2475,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...


In [9]:
#y lo que aplico a este otro dataframe es la logica de separacion para json, que estos estaban raros por ese \n
dict_df2 = df_objects[['dependant_types', 'line_items', 'discounts', 'line_item_tiers', 'linked_payments',
'dunning_attempts', 'applied_credits', 'adjustment_credit_notes', 'issued_credit_notes', 'linked_orders', 'billing_address',
'site_details_at_creation', 'tax_origin']]

In [10]:
dict_df2

,dependant_types,line_items,discounts,line_item_tiers,linked_payments,dunning_attempts,applied_credits,adjustment_credit_notes,issued_credit_notes,linked_orders,billing_address,site_details_at_creation,tax_origin
0,{},"[{\n ""id"": ""li_6olL2UKBV7cg22wd"",\n ""dat...",None,"[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_AzqB1XUMIWXep30Ov"",\n ...","[{\n ""created_at"": 1722567797,\n ""attemp...",[],[],[],[],"{\n ""company"": ""PARRA E HIJOS S.A."",\n ""...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
1,{},"[{\n ""id"": ""li_Azz6aoUKC1bN11M2p"",\n ""da...","[{\n ""object"": ""discount"",\n ""entity_typ...","[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_16CXuLULLoKeC2h8c"",\n ...","[{\n ""created_at"": 1722568032,\n ""attemp...",[],[],[],[],"{\n ""company"": ""Nuke"",\n ""country"": ""AR""...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
2,{},"[{\n ""id"": ""li_AzyjxKUKC78gO1LOQ"",\n ""da...",None,"[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_16BW6dUJLpJrc1Cpj"",\n ...",[],[],[],[],[],"{\n ""first_name"": ""Martin"",\n ""last_name...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
3,{},"[{\n ""id"": ""li_6olL2UKCAuip39NE"",\n ""dat...",None,"[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_AzqBCbUKy7RvD2RIN"",\n ...","[{\n ""created_at"": 1722567951,\n ""attemp...",[],[],[],[],"{\n ""company"": ""http://www.perkinelmer.com....","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
4,{},"[{\n ""id"": ""li_6olL2UKCDiUk3E1u"",\n ""dat...",None,"[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_6olL2UKCDiWS3E1z"",\n ...",[],[],[],[],[],"{\n ""first_name"": ""Francisco"",\n ""last_n...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,{},"[{\n ""id"": ""li_16BdL2UN5VwgXKJF5"",\n ""da...","[{\n ""object"": ""discount"",\n ""entity_typ...","[{\n ""starting_unit"": 1,\n ""ending_unit""...",[],[],[],[],[],[],"{\n ""company"": ""Renault Directa"",\n ""cou...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
2473,{},"[{\n ""id"": ""li_16BdL2UN5Vwo1KJFY"",\n ""da...","[{\n ""object"": ""discount"",\n ""entity_typ...","[{\n ""starting_unit"": 1,\n ""ending_unit""...",[],[],[],[],[],[],"{\n ""company"": ""tuprode.com"",\n ""country...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
2474,{},"[{\n ""id"": ""li_16BdL2UN5VvwOKJC3"",\n ""da...","[{\n ""object"": ""discount"",\n ""entity_typ...","[{\n ""starting_unit"": 1,\n ""ending_unit""...",[],[],[],[],"[{\n ""cn_id"": ""CN-7066"",\n ""cn_reason_co...",[],"{\n ""first_name"": ""Mica"",\n ""last_name"":...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
2475,{},"[{\n ""id"": ""li_16BdL2UN5VvzbKJCO"",\n ""da...","[{\n ""object"": ""discount"",\n ""entity_typ...","[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_16CXx1UO5eUVR3dvn"",\n ...","[{\n ""created_at"": 1725246813,\n ""attemp...",[],[],[],[],"{\n ""first_name"": ""Lucas"",\n ""last_name""...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"


In [11]:
#aqui estan las funciones para dividir los campos de diccionario

# Función para desanidar el diccionario en cada fila de una columna específica
def flatten_json_column(df, column_name):
    # Verificar si la columna es de tipo diccionario o tiene JSON
    if df[column_name].apply(lambda x: isinstance(x, dict)).all():
        # Desanidar la columna 'column_name'
        df_expanded = pd.json_normalize(df[column_name])
        
        # Concatenar las nuevas columnas con el DataFrame original, eliminando la columna original desanidada
        # Agregar sufijos a las columnas que se solapan.
        df = df.drop(columns=[column_name]).join(df_expanded, rsuffix=f'_{column_name}_expanded')
    
    return df

# Función para desanidar todas las columnas de tipo diccionario o JSON en un DataFrame
def flatten_all_json_columns(df):
    # Usar .map en lugar de .applymap para verificar columnas con diccionarios
    columns_to_flatten = df.columns[df.apply(lambda col: col.map(lambda x: isinstance(x, dict)).any())]
    for column in columns_to_flatten:
        df = flatten_json_column(df, column)
    
    return df

# Aplica la función a todas las columnas que contienen diccionarios o JSON
df_flattened_all = flatten_all_json_columns(dict_df)

# Imprimir el nuevo DataFrame con todas las columnas JSON desanidadas
df_flattened_all


,id,po_number,customer_id,subscription_id,recurring,status,price_type,date,due_date,net_term_days,...,adjustment_credit_notes_sub_types_expanded,issued_credit_notes_sub_types_expanded,linked_orders_sub_types_expanded,notes_sub_types_expanded,shipping_address,statement_descriptor,billing_address,einvoice,site_details_at_creation,tax_origin
0,185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,tax_exclusive,1722499680,1722499680,0,...,<class 'chargebee.models.invoice.Invoice.Adjus...,<class 'chargebee.models.invoice.Invoice.Issue...,<class 'chargebee.models.invoice.Invoice.Linke...,<class 'chargebee.models.invoice.Invoice.Note'>,<class 'chargebee.models.invoice.Invoice.Shipp...,<class 'chargebee.models.invoice.Invoice.State...,<class 'chargebee.models.invoice.Invoice.Billi...,<class 'chargebee.models.invoice.Invoice.Einvo...,<class 'chargebee.models.invoice.Invoice.SiteD...,<class 'chargebee.models.invoice.Invoice.TaxOr...
1,185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,tax_exclusive,1722507420,1722507420,0,...,<class 'chargebee.models.invoice.Invoice.Adjus...,<class 'chargebee.models.invoice.Invoice.Issue...,<class 'chargebee.models.invoice.Invoice.Linke...,<class 'chargebee.models.invoice.Invoice.Note'>,<class 'chargebee.models.invoice.Invoice.Shipp...,<class 'chargebee.models.invoice.Invoice.State...,<class 'chargebee.models.invoice.Invoice.Billi...,<class 'chargebee.models.invoice.Invoice.Einvo...,<class 'chargebee.models.invoice.Invoice.SiteD...,<class 'chargebee.models.invoice.Invoice.TaxOr...
2,185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,True,paid,tax_exclusive,1722508740,1722508740,0,...,<class 'chargebee.models.invoice.Invoice.Adjus...,<class 'chargebee.models.invoice.Invoice.Issue...,<class 'chargebee.models.invoice.Invoice.Linke...,<class 'chargebee.models.invoice.Invoice.Note'>,<class 'chargebee.models.invoice.Invoice.Shipp...,<class 'chargebee.models.invoice.Invoice.State...,<class 'chargebee.models.invoice.Invoice.Billi...,<class 'chargebee.models.invoice.Invoice.Einvo...,<class 'chargebee.models.invoice.Invoice.SiteD...,<class 'chargebee.models.invoice.Invoice.TaxOr...
3,185395,Argentina - Transferencia Bancaria,5cb4d5d0e4b0bd6bed7869fa,16CY2RUEdBvEE1YCI,True,paid,tax_exclusive,1722509640,1722509640,0,...,<class 'chargebee.models.invoice.Invoice.Adjus...,<class 'chargebee.models.invoice.Invoice.Issue...,<class 'chargebee.models.invoice.Invoice.Linke...,<class 'chargebee.models.invoice.Invoice.Note'>,<class 'chargebee.models.invoice.Invoice.Shipp...,<class 'chargebee.models.invoice.Invoice.State...,<class 'chargebee.models.invoice.Invoice.Billi...,<class 'chargebee.models.invoice.Invoice.Einvo...,<class 'chargebee.models.invoice.Invoice.SiteD...,<class 'chargebee.models.invoice.Invoice.TaxOr...
4,185396,Chile,5b47de04e4b03c859495a0f7,Izy9TIBR04obyPV06,True,paid,tax_exclusive,1722510308,1722510308,0,...,<class 'chargebee.models.invoice.Invoice.Adjus...,<class 'chargebee.models.invoice.Invoice.Issue...,<class 'chargebee.models.invoice.Invoice.Linke...,<class 'chargebee.models.invoice.Invoice.Note'>,<class 'chargebee.models.invoice.Invoice.Shipp...,<class 'chargebee.models.invoice.Invoice.State...,<class 'chargebee.models.invoice.Invoice.Billi...,<class 'chargebee.models.invoice.Invoice.Einvo...,<class 'chargebee.models.invoice.Invoice.SiteD...,<class 'chargebee.models.invoice.Invoice.TaxOr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,187908,Argentina - Transferencia Bancaria (Leadaki),5f3da9f3b68252002aaac99e,AzZsu3UBeV4iY8HEC,True,paid,tax_exclusive,1725159600,1725159600,0,...,<class 'chargebee.models.invoice.Invoice.Adjus...,<class 'chargebee.models.invoice.Invoice.Issue...,<class 'chargebee.models.invoice.Invoice.Linke...,<class 'chargebee.models.invoice.Invoice.Note'>,<class 'chargebee.models.invoice.Invoice.Shipp...,<class 'chargebee

In [12]:
#y aqui las columnas que dividen los campos json
import json

# Función para desanidar columnas que contienen listas de diccionarios (JSON)
def flatten_list_column(df, column_name):
    # Verificamos si las filas de la columna contienen listas de diccionarios
    df[column_name] = df[column_name].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
    
    # Expandir la lista de diccionarios en nuevas filas
    df_expanded = df[column_name].explode().reset_index(drop=True)
    
    # Desanidar los diccionarios en nuevas columnas
    df_expanded = pd.json_normalize(df_expanded)
    
    # Concatenar el dataframe original con las columnas desanidadas
    df = df.drop(columns=[column_name]).join(df_expanded, rsuffix=f'_{column_name}_expanded')
    
    return df

# Función para desanidar todas las columnas que contienen listas de diccionarios en un DataFrame
def flatten_all_list_columns(df):
    # Verificar si alguna columna contiene listas de diccionarios
    columns_to_flatten = df.columns[df.apply(lambda col: col.map(lambda x: isinstance(x, list) and isinstance(x[0], dict) if isinstance(x, list) and len(x) > 0 else False).any())]
    
    # Desanidar cada columna que contenga listas de diccionarios
    for column in columns_to_flatten:
        df = flatten_list_column(df, column)
    
    return df

# Supongamos que tu dataframe se llama df_flattened_all, ya que es el resultado de desanidar los diccionarios.
# Ahora desanidamos las columnas que contienen listas de objetos (JSON).
df_final = flatten_all_list_columns(df_flattened_all)

# Mostrar el DataFrame con las listas desanidadas
df_final

,id,po_number,customer_id,subscription_id,recurring,status,price_type,date,due_date,net_term_days,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded
0,185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,tax_exclusive,1722499680,1722499680,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,tax_exclusive,1722507420,1722507420,0,...,NaN,line_item_discount,li_Azz6aoUKC1bN81M2q,item_level_coupon,2000.0,Migra2024Premium2Users,Migra2024Premium2Users,NaN,NaN,NaN
2,185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,True,paid,tax_exclusive,1722508740,1722508740,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,185395,Argentina - Transferencia Bancaria,5cb4d5d0e4b0bd6bed7869fa,16CY2RUEdBvEE1YCI,True,paid,tax_exclusive,1722509640,1722509640,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,185396,Chile,5b47de04e4b03c859495a0f7,Izy9TIBR04obyPV06,True,paid,tax_exclusive,1722510308,1722510308,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,187908,Argentina - Transferencia Bancaria (Leadaki),5f3da9f3b68252002aaac99e,AzZsu3UBeV4iY8HEC,True,paid,tax_exclusive,1725159600,1725159600,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2473,187910,Argentina - Transferencia Bancaria (Leadaki),5a71ba83e4b07b95e31fc272,AzqBxKUBeW38u8EGG,True,paid,tax_exclusive,1725159600,1725159600,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2474,187892,Argentina - Transferencia Bancaria (Leadaki),57bddbafe4b08f5c877fe039,169v0eUBcr4Bg73Qq,True,paid,tax_exclusive,1725159600,1725159600,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2475,187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,tax_exclusive,1725159600,1725159600,0,...,100.0,line_item_discount,li_16BdL2UN4mcBRIeat,item_level_coupon,700.0,MIGRA2024STARTER1USER,MIGRA2024STARTER1USER,NaN,NaN,NaN


In [13]:
#Le cambie la region a argentina utc tal y como lo tuvimos que hacer con panella cuando los traiamos de BigQuery antes
import pytz
argentina_tz = pytz.timezone('America/Argentina/Buenos_Aires')
df_final['date_from'] = pd.to_datetime(df_final['date_from'], unit='s', utc=True).dt.tz_convert(argentina_tz)
df_final['date_to'] = pd.to_datetime(df_final['date_to'], unit='s', utc=True).dt.tz_convert(argentina_tz)
df_final['applied_at'] = pd.to_datetime(df_final['applied_at'], unit='s', utc=True).dt.tz_convert(argentina_tz)
df_final['generated_at'] = pd.to_datetime(df_final['generated_at'], unit='s', utc=True).dt.tz_convert(argentina_tz)
df_final['updated_at'] = pd.to_datetime(df_final['updated_at'], unit='s', utc=True).dt.tz_convert(argentina_tz)
df_final['paid_at'] = pd.to_datetime(df_final['paid_at'], unit='s', utc=True).dt.tz_convert(argentina_tz)
df_final['due_date'] = pd.to_datetime(df_final['due_date'], unit='s', utc=True).dt.tz_convert(argentina_tz)
df_final['date'] = pd.to_datetime(df_final['date'], unit='s', utc=True).dt.tz_convert(argentina_tz)

In [14]:
df_final

,id,po_number,customer_id,subscription_id,recurring,status,price_type,date,due_date,net_term_days,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded
0,185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,tax_exclusive,2024-08-01 05:08:00-03:00,2024-08-01 05:08:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,tax_exclusive,2024-08-01 07:17:00-03:00,2024-08-01 07:17:00-03:00,0,...,NaN,line_item_discount,li_Azz6aoUKC1bN81M2q,item_level_coupon,2000.0,Migra2024Premium2Users,Migra2024Premium2Users,NaN,NaN,NaN
2,185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,True,paid,tax_exclusive,2024-08-01 07:39:00-03:00,2024-08-01 07:39:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,185395,Argentina - Transferencia Bancaria,5cb4d5d0e4b0bd6bed7869fa,16CY2RUEdBvEE1YCI,True,paid,tax_exclusive,2024-08-01 07:54:00-03:00,2024-08-01 07:54:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,185396,Chile,5b47de04e4b03c859495a0f7,Izy9TIBR04obyPV06,True,paid,tax_exclusive,2024-08-01 08:05:08-03:00,2024-08-01 08:05:08-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,187908,Argentina - Transferencia Bancaria (Leadaki),5f3da9f3b68252002aaac99e,AzZsu3UBeV4iY8HEC,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2473,187910,Argentina - Transferencia Bancaria (Leadaki),5a71ba83e4b07b95e31fc272,AzqBxKUBeW38u8EGG,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2474,187892,Argentina - Transferencia Bancaria (Leadaki),57bddbafe4b08f5c877fe039,169v0eUBcr4Bg73Qq,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2475,187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,line_item_discount,li_16BdL2UN4mcBRIeat,item_level_coupon,700.0,MIGRA2024STARTER1USER,MIGRA2024STARTER1USER,NaN,NaN,NaN


In [156]:
#ordernar por id de manera descendente
order = df_final.sort_values(by = 'id', ascending=True)
order

,id,po_number,customer_id,subscription_id,recurring,status,price_type,date,due_date,net_term_days,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded
0,185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,tax_exclusive,2024-08-01 05:08:00-03:00,2024-08-01 05:08:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,tax_exclusive,2024-08-01 07:17:00-03:00,2024-08-01 07:17:00-03:00,0,...,NaN,line_item_discount,li_Azz6aoUKC1bN81M2q,item_level_coupon,2000.0,Migra2024Premium2Users,Migra2024Premium2Users,NaN,NaN,NaN
2,185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,True,paid,tax_exclusive,2024-08-01 07:39:00-03:00,2024-08-01 07:39:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,185395,Argentina - Transferencia Bancaria,5cb4d5d0e4b0bd6bed7869fa,16CY2RUEdBvEE1YCI,True,paid,tax_exclusive,2024-08-01 07:54:00-03:00,2024-08-01 07:54:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,185396,Chile,5b47de04e4b03c859495a0f7,Izy9TIBR04obyPV06,True,paid,tax_exclusive,2024-08-01 08:05:08-03:00,2024-08-01 08:05:08-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,187910,Argentina - Transferencia Bancaria (Leadaki),5a71ba83e4b07b95e31fc272,AzqBxKUBeW38u8EGG,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2467,187911,Argentina - Transferencia Bancaria (Leadaki),5727680de4b06a6e4222f0c8,AzyhzvUBeSCea8nZZ,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2468,187912,Argentina - Transferencia Bancaria (Leadaki),53c42a63e4b08c5d83b878ff,AzqBxKUBeN2WG88Ao,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2460,187913,Argentina - Transferencia Bancaria (Leadaki),536109b2e4b0ae650c483374,16CY4nUBd1nGm7OLS,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
print_column_types(order)

id 	 object
po_number 	 object
customer_id 	 object
subscription_id 	 object
recurring 	 bool
status 	 object
price_type 	 object
date 	 datetime64[ns, America/Argentina/Buenos_Aires]
due_date 	 datetime64[ns, America/Argentina/Buenos_Aires]
net_term_days 	 int64
exchange_rate 	 float64
total 	 int64
amount_paid 	 int64
amount_adjusted 	 int64
write_off_amount 	 int64
credits_applied 	 int64
amount_due 	 int64
paid_at 	 datetime64[ns, America/Argentina/Buenos_Aires]
dunning_status 	 object
updated_at 	 datetime64[ns, America/Argentina/Buenos_Aires]
resource_version 	 int64
deleted 	 bool
object 	 object
first_invoice 	 bool
amount_to_collect 	 int64
round_off_amount 	 int64
has_advance_charges 	 bool
currency_code 	 object
base_currency_code 	 object
generated_at 	 datetime64[ns, America/Argentina/Buenos_Aires]
is_gifted 	 bool
term_finalized 	 bool
channel 	 object
tax 	 int64
sub_total 	 int64
linked_orders 	 object
billing_address.company 	 object
billing_address.country 	 object
bi

In [ ]:
#a partir de aqui empieza lo de Odoo

In [17]:
#primero con las pruebas de insercion en sales.order


odoo.create('sale.order', [{
                            'campaign_id': False,
                            'source_id': False,
                            'medium_id': False,
                            'activity_ids': [],
                            'activity_state': True,
                            'partner_id': 11, 
                            'company_id': 1,
                            'website_id': 1,
                            }])


356

In [17]:
# aqui yo buscaba antes un id recien creado por el codigo anterior
sale_order_345 = odoo.execute('sale.order', 'search_read', [[['id', '=', 354]], [], 0, 1])
print(sale_order_345)

[{'id': 354, 'campaign_id': False, 'source_id': False, 'medium_id': False, 'activity_ids': [], 'activity_state': False, 'activity_user_id': False, 'activity_type_id': False, 'activity_type_icon': False, 'activity_date_deadline': False, 'my_activity_date_deadline': False, 'activity_summary': False, 'activity_exception_decoration': False, 'activity_exception_icon': False, 'activity_calendar_event_id': False, 'message_is_follower': True, 'message_follower_ids': [1243], 'message_partner_ids': [121], 'message_ids': [2745], 'has_message': True, 'message_needaction': False, 'message_needaction_counter': 0, 'message_has_error': False, 'message_has_error_counter': 0, 'message_attachment_count': 0, 'rating_ids': [], 'website_message_ids': [], 'message_has_sms_error': False, 'access_url': '/my/orders/354', 'access_token': False, 'access_warning': '', 'name': 'V 0001-00000342', 'company_id': [1, 'REGEX LLC'], 'partner_id': [11, 'Gemini Furniture'], 'state': 'draft', 'locked': False, 'client_order_

In [18]:
#aqui hacemos el llamado a las invoices de clientes en Odoo
odf = odoo.execute('account.move', 'search_read', [[['id', '>', 1]], 0, 20])
dfc = pd.DataFrame(odf)
dfc

,id,campaign_id,source_id,medium_id,is_tax_computed_externally,is_avatax,sequence_prefix,sequence_number,activity_ids,activity_state,...,l10n_ar_afip_asoc_period_start,l10n_ar_afip_asoc_period_end,l10n_ar_boarding_permission_ids,timesheet_ids,timesheet_count,timesheet_encode_uom_id,timesheet_total_duration,document_type_internal_type,invoice_validation_type,l10n_ar_edi_warning
0,35782,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
1,35781,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2,35780,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
3,35779,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
4,35778,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2286,19,False,False,False,False,False,BNK1/2024/,3,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2287,18,False,False,False,False,False,BNK1/2024/,2,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2288,17,False,False,False,False,False,BNK1/2024/,1,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2289,11,False,False,False,False,False,RBILL/2023/05/,1,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False


In [19]:
#este nada mas es una busqueda de una factura en especifico por medio del campo name, 
# que es el que aparece en la interfaz grafica de odoo
dfc.loc[dfc['name'] == 'BNK1/2024/00007']

,id,campaign_id,source_id,medium_id,is_tax_computed_externally,is_avatax,sequence_prefix,sequence_number,activity_ids,activity_state,...,l10n_ar_afip_asoc_period_start,l10n_ar_afip_asoc_period_end,l10n_ar_boarding_permission_ids,timesheet_ids,timesheet_count,timesheet_encode_uom_id,timesheet_total_duration,document_type_internal_type,invoice_validation_type,l10n_ar_edi_warning
2282,23,False,False,False,False,False,BNK1/2024/,7,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False


In [21]:
#de la funcion que cree al principio se la aplique al dataframe de las invoices de odoo 
# para ver el tipo de datos de cada campo del modelo account.move
print_column_types(dfc)

id 	 int64
campaign_id 	 bool
source_id 	 bool
medium_id 	 bool
is_tax_computed_externally 	 bool
is_avatax 	 bool
sequence_prefix 	 object
sequence_number 	 int64
activity_ids 	 object
activity_state 	 object
activity_user_id 	 object
activity_type_id 	 object
activity_type_icon 	 object
activity_date_deadline 	 object
my_activity_date_deadline 	 bool
activity_summary 	 object
activity_exception_decoration 	 bool
activity_exception_icon 	 bool
activity_calendar_event_id 	 bool
access_url 	 object
access_token 	 bool
access_warning 	 object
message_is_follower 	 bool
message_follower_ids 	 object
message_partner_ids 	 object
message_ids 	 object
has_message 	 bool
message_needaction 	 bool
message_needaction_counter 	 int64
message_has_error 	 bool
message_has_error_counter 	 int64
message_attachment_count 	 int64
rating_ids 	 object
website_message_ids 	 object
message_has_sms_error 	 bool
message_main_attachment_id 	 object
name 	 object
ref 	 object
date 	 object
state 	 object
move

In [22]:
#aqui hice el llamado de los campos de tipo object del dataframe para ver los campos que ocupan el equivalente a un valor de moneda
#lo hice para ir viendo los campos que mas adelante debo comparar con los de Chargebee y hacer la migracion correctamente
floats = dfc.select_dtypes(include=['float64'])
floats

,amount_untaxed,amount_tax,amount_total,amount_residual,amount_untaxed_signed,amount_tax_signed,amount_total_signed,amount_total_in_currency_signed,amount_residual_signed,quick_edit_total_amount,partner_credit,amount_paid,asset_remaining_value,asset_depreciated_value,depreciation_value,l10n_ar_currency_rate,computed_currency_rate
0,28.75,0.00,28.75,28.75,28.75,0.00,28.75,28.75,28.75,0.0,28.75,0.0,0.0,0.0,0.0,0.0,1.0
1,28.75,0.00,28.75,28.75,28.75,0.00,28.75,28.75,28.75,0.0,28.75,0.0,0.0,0.0,0.0,0.0,1.0
2,28.75,0.00,28.75,28.75,28.75,0.00,28.75,28.75,28.75,0.0,28.75,0.0,0.0,0.0,0.0,0.0,1.0
3,28.75,0.00,28.75,28.75,28.75,0.00,28.75,28.75,28.75,0.0,28.75,0.0,0.0,0.0,0.0,0.0,1.0
4,28.75,0.00,28.75,28.75,28.75,0.00,28.75,28.75,28.75,0.0,28.75,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2286,0.00,0.00,32.58,0.00,0.00,0.00,32.58,32.58,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.0
2287,0.00,0.00,1275.00,0.00,0.00,0.00,1275.00,1275.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.0
2288,0.00,0.00,5103.00,0.00,0.00,0.00,5103.00,5103.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.0
2289,541.10,81.17,622.27,622.27,541.10,81.17,622.27,622.27,622.27,0.0,622.27,0.0,0.0,0.0,0.0,0.0,1.0


In [23]:
#este codigo me muestra en un dataframe solamente los campos de tipo float de account.move pero sin excluir el id de la misma
# por eso le hago un concat con el original para que conserve el id
if 'id' in dfc.columns:
    floats_with_id = pd.concat([floats, dfc[['id', 'name']]], axis=1)
else:
    print("La columna 'id' no existe en el DataFrame original.")

floats_with_id

,amount_untaxed,amount_tax,amount_total,amount_residual,amount_untaxed_signed,amount_tax_signed,amount_total_signed,amount_total_in_currency_signed,amount_residual_signed,quick_edit_total_amount,partner_credit,amount_paid,asset_remaining_value,asset_depreciated_value,depreciation_value,l10n_ar_currency_rate,computed_currency_rate,id,name
0,28.75,0.00,28.75,28.75,28.75,0.00,28.75,28.75,28.75,0.0,28.75,0.0,0.0,0.0,0.0,0.0,1.0,35782,/
1,28.75,0.00,28.75,28.75,28.75,0.00,28.75,28.75,28.75,0.0,28.75,0.0,0.0,0.0,0.0,0.0,1.0,35781,/
2,28.75,0.00,28.75,28.75,28.75,0.00,28.75,28.75,28.75,0.0,28.75,0.0,0.0,0.0,0.0,0.0,1.0,35780,/
3,28.75,0.00,28.75,28.75,28.75,0.00,28.75,28.75,28.75,0.0,28.75,0.0,0.0,0.0,0.0,0.0,1.0,35779,/
4,28.75,0.00,28.75,28.75,28.75,0.00,28.75,28.75,28.75,0.0,28.75,0.0,0.0,0.0,0.0,0.0,1.0,35778,/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2286,0.00,0.00,32.58,0.00,0.00,0.00,32.58,32.58,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.0,19,BNK1/2024/00003
2287,0.00,0.00,1275.00,0.00,0.00,0.00,1275.00,1275.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.0,18,BNK1/2024/00002
2288,0.00,0.00,5103.00,0.00,0.00,0.00,5103.00,5103.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.0,17,BNK1/2024/00001
2289,541.10,81.17,622.27,622.27,541.10,81.17,622.27,622.27,622.27,0.0,622.27,0.0,0.0,0.0,0.0,0.0,1.0,11,RBILL/2023/05/0001


In [24]:
#aqui fue en donde vi como es que se almacena el registro de partner_id (que es un array)
sub_dfc = dfc['partner_id']
sub_dfc

0       [26253, 66b5611d83457860cafb49eb]
1       [26195, 66b5408baa08501f9aa2503a]
2       [25886, 66b3eb155ac43769d434ca95]
3       [24834, 66b2e905e699225d840d64f8]
4       [25856, 66b28d2b796a601e95fe096d]
                      ...                
2286                                False
2287                 [14, Azure Interior]
2288                                False
2289                 [14, Azure Interior]
2290                 [14, Azure Interior]
Name: partner_id, Length: 2291, dtype: object

In [25]:
# Leer todas las facturas (sin establecer un dominio específico) todo para comprobar que la insercion fue exitosa
invoices = odoo.search_read('account.move', [], ['id', 'name', 'date', 'amount_total', 'partner_id'])

customers_invoice = pd.DataFrame(invoices)
customers_invoice


,id,name,date,amount_total,partner_id
0,35802,/,2024-10-04,28.75,"[26428, 66d1e1c49112e41a78b90d4c]"
1,35801,/,2024-10-04,28.75,"[26425, 66d0a6efa9c6e736847a56e6]"
2,35800,/,2024-10-04,28.75,"[26363, 66cf6f0374ecf760b7a5b258]"
3,35799,/,2024-10-04,28.75,"[26366, 66cf6e434427b90002f38734]"
4,35798,/,2024-10-04,28.75,"[26390, 66ccdda3b1c5d119cf1a2412]"
5,35797,/,2024-10-04,28.75,"[26400, 66ca541d8bda3e6c4bcc53f7]"
6,35796,/,2024-10-04,28.75,"[25902, 66c68ef9356e9a1968c3948f]"
7,35795,/,2024-10-04,28.75,"[26258, 66c67483d521095ebb508479]"
8,35794,/,2024-10-04,28.75,"[25965, 66c671f5042e05078961406a]"
9,35793,/,2024-10-04,28.75,"[25986, 66c62482042e0507896134b7]"


In [26]:
#llame a todas las invoice otra vez pero para separar los campos del array(al final no sirvio porque solamente fue una operacion 
# de lectura :()
i = odoo.search_read('account.move', [], [], limit=None)
inv = pd.DataFrame(i)
inv

,id,campaign_id,source_id,medium_id,is_tax_computed_externally,is_avatax,sequence_prefix,sequence_number,activity_ids,activity_state,...,l10n_ar_afip_asoc_period_start,l10n_ar_afip_asoc_period_end,l10n_ar_boarding_permission_ids,timesheet_ids,timesheet_count,timesheet_encode_uom_id,timesheet_total_duration,document_type_internal_type,invoice_validation_type,l10n_ar_edi_warning
0,35802,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
1,35801,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2,35800,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
3,35799,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
4,35798,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2307,19,False,False,False,False,False,BNK1/2024/,3,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2308,18,False,False,False,False,False,BNK1/2024/,2,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2309,17,False,False,False,False,False,BNK1/2024/,1,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2310,11,False,False,False,False,False,RBILL/2023/05/,1,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False


In [27]:
#solo mostrar el campo de partner_id para ver como quedan los customer_id de Chargebee  en Odoo
inv[['id', 'partner_id', 'name', 'invoice_line_ids']]

,id,partner_id,name,invoice_line_ids
0,35802,"[26428, 66d1e1c49112e41a78b90d4c]",/,[40390]
1,35801,"[26425, 66d0a6efa9c6e736847a56e6]",/,[40392]
2,35800,"[26363, 66cf6f0374ecf760b7a5b258]",/,[40394]
3,35799,"[26366, 66cf6e434427b90002f38734]",/,[40396]
4,35798,"[26390, 66ccdda3b1c5d119cf1a2412]",/,[40398]
...,...,...,...,...
2307,19,False,BNK1/2024/00003,"[57, 58]"
2308,18,"[14, Azure Interior]",BNK1/2024/00002,"[55, 56]"
2309,17,False,BNK1/2024/00001,"[53, 54]"
2310,11,"[14, Azure Interior]",RBILL/2023/05/0001,[20]


In [28]:
#realice el llamado del campo del customer de Chargebee para ingresarlo en res.partner
customerdf = order['customer_id']
customerdf

0       5acdf7d8e4b014cba3b1724b
1       5aa291c2e4b0c65fcc798ce8
2       5ad08c66e4b01bf0eefaf887
3       5cb4d5d0e4b0bd6bed7869fa
4       5b47de04e4b03c859495a0f7
                  ...           
2473    5a71ba83e4b07b95e31fc272
2467    5727680de4b06a6e4222f0c8
2468    53c42a63e4b08c5d83b878ff
2460    536109b2e4b0ae650c483374
2476    5a0c58c9e4b031a2c96d319c
Name: customer_id, Length: 2477, dtype: object

In [165]:
#como los customers que vamos a ingresar estaban duplicados en el llamado, pues elimine los duplicados ya que a partir de este dataframe 
#hare las inserciones en el modelo res.partner
customers = customerdf.drop_duplicates()
customers

0       5acdf7d8e4b014cba3b1724b
1       5aa291c2e4b0c65fcc798ce8
2       5ad08c66e4b01bf0eefaf887
3       5cb4d5d0e4b0bd6bed7869fa
4       5b47de04e4b03c859495a0f7
                  ...           
2473    5a71ba83e4b07b95e31fc272
2467    5727680de4b06a6e4222f0c8
2468    53c42a63e4b08c5d83b878ff
2460    536109b2e4b0ae650c483374
2476    5a0c58c9e4b031a2c96d319c
Name: customer_id, Length: 2267, dtype: object

In [154]:
#busque los res.partner que es en donde se van a almacenar los customer de Chargebee

i2 = odoo.search_read('res.partner', [], [], limit=None)  # Establecer limit=None para obtener todos los registros
partner = pd.DataFrame(i2)

partner

KeyboardInterrupt: 

In [31]:
#deshacerme de los nombres duplicados del modelo res.partner
all_partners=partner.drop_duplicates('name')
all_partners

,id,avatax_unique_code,website_id,website_published,is_published,can_publish,website_url,message_is_follower,message_follower_ids,message_partner_ids,...,l10n_es_edi_facturae_ac_role_type_ids,l10n_es_edi_facturae_ac_physical_gln,l10n_es_edi_facturae_ac_logical_operational_point,sale_type,subscription_count,last_website_so_id,l10n_cl_delivery_guide_price,arba_alicuot_ids,drei,default_regimen_ganancias_id
0,86,Contact 86,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
1,87,Contact 87,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
2,67,Contact 67,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
3,25762,Contact 25762,False,False,False,True,#,True,[60258],[121],...,[],False,False,False,0,False,sale_order,[],False,False
4,26495,Contact 26495,False,False,False,True,#,True,[60991],[121],...,[],False,False,False,0,False,sale_order,[],False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2386,69,Contact 69,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
2387,77,Contact 77,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
2388,63,Contact 63,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
2389,76,Contact 76,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False


In [32]:
#visualizo el id y elcampo en donde se muestra el nombre del customer
i3 = partner[['name', 'id']]

In [33]:
i3

,name,id
0,(AR) Exento,86
1,(AR) Monotributista,87
2,0h3n-rijaune@yahoo.example.com,67
3,16CXyqUDJkqSi1S5N,25762
4,535680f7e4b03a7c45f66703,26495
...,...,...
2386,the.jose@gmail.example.com,69
2387,the.real.shane@gmail.example.com,77
2388,thing.thang.thong@gmail.example.com,63
2389,tina.turner@gmail.example.com,76


In [108]:
#luego ejecute una insercion en ese modelo y funciono exitosamente (el campo que vi que podia llenar con el string de un customer_id)
#era ese campo name de todos los que tiene el dataframe


#odoo.create('res.partner', [{
#    'name': '5dde8435e4b098a27bbd4def'
#}])

465

In [35]:
#veo los tipos de datos del dataframe de los partners solo con nombre y id
print_column_types(i3)

name 	 object
id 	 int64


In [36]:
#genere un dataframe sobre los campos con tipo de dato object que es un equivalente a un string y poder encontrar que campo habia que
# llenar para que me se pudiera ver el nombre del partner en la interfaz grafica
many = partner.select_dtypes(include=['object'])
many

,avatax_unique_code,website_url,message_follower_ids,message_partner_ids,message_ids,rating_ids,website_message_ids,email_normalized,activity_ids,image_1920,...,gross_income_jurisdiction_ids,imp_ganancias_padron,actividades_padron,impuestos_padron,l10n_cl_dte_email,l10n_es_edi_facturae_ac_center_code,l10n_es_edi_facturae_ac_role_type_ids,l10n_cl_delivery_guide_price,arba_alicuot_ids,default_regimen_ganancias_id
0,Contact 86,#,[],[],[1115],[],[],info@exento.example.com,[],iVBORw0KGgoAAAANSUhEUgAAAcIAAAB4CAYAAABhPvLiAA...,...,[],False,[],[],False,False,[],sale_order,[],False
1,Contact 87,#,[],[],"[1382, 1126]",[],[],info@example.com,[],iVBORw0KGgoAAAANSUhEUgAAAcIAAAB4CAYAAABhPvLiAA...,...,[],False,[],[],False,False,[],sale_order,[],False
2,Contact 67,#,[],[],[480],[],[],0h3n-rijaune@yahoo.example.com,[],False,...,[],False,[],[],False,False,[],sale_order,[],False
3,Contact 25762,#,[60258],[121],[67846],[],[],False,[],False,...,[],False,[],[],False,False,[],sale_order,[],False
4,Contact 26495,#,[60991],[121],[68579],[],[],False,[],False,...,[],False,[],[],False,False,[],sale_order,[],False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2386,Contact 69,#,[],[],[484],[],[],the.jose@gmail.example.com,[],False,...,[],False,[],[],False,False,[],sale_order,[],False
2387,Contact 77,#,[],[],[500],[],[],the.real.shane@gmail.example.com,[],False,...,[],False,[],[],False,False,[],sale_order,[],False
2388,Contact 63,#,[],[],[472],[],[],thing.thang.thong@gmail.example.com,[],False,...,[],False,[],[],False,False,[],sale_order,[],False
2389,Contact 76,#,[],[],[498],[],[],tina.turner@gmail.example.com,[],False,...,[],False,[],[],False,False,[],sale_order,[],False


In [37]:
#una vez encontrado lo llamo del dataframe anterior
many['name']

0                               (AR) Exento
1                       (AR) Monotributista
2            0h3n-rijaune@yahoo.example.com
3                         16CXyqUDJkqSi1S5N
4                  535680f7e4b03a7c45f66703
                       ...                 
2386             the.jose@gmail.example.com
2387       the.real.shane@gmail.example.com
2388    thing.thang.thong@gmail.example.com
2389          tina.turner@gmail.example.com
2390             yin.lee@wechat.example.com
Name: name, Length: 2391, dtype: object

In [212]:
#empiezo a generar el codigo para hacer una insercion de los customers de Chargebee en Odoo
# Inicializar una lista para almacenar el historial de inserciones
insertion_history = []

# Función para insertar nuevos registros en res.partner
def insert_new_partners(customers, execution_id):
    # Leer todos los registros actuales del modelo res.partner
    existing_partners = odoo.search_read('res.partner', [], ['name'])
    
    # Extraer los nombres de los partners existentes
    existing_partner_names = {partner['name'] for partner in existing_partners}
    
    # Obtener la lista de nombres de customers del DataFrame
    customer_ids = customers.tolist()
    
    # Crear una lista de diccionarios para los nuevos registros a insertar
    # Solo insertar aquellos que no existen en el modelo res.partner
    records_to_insert = [{'name': customer_id} for customer_id in customer_ids if customer_id not in existing_partner_names]
    
    # Realizar la inserción masiva uno por uno para evitar sobrecarga en la llamada a Odoo.create
    for record in records_to_insert:
        odoo.create('res.partner', [record])
    
    # Guardar los nuevos registros en el historial con un ID de ejecución
    insertion_history.append({
        'execution_id': execution_id,
        'new_records': records_to_insert
    })
    
    # Mostrar cuántos registros nuevos fueron insertados
    print(f"Se insertaron {len(records_to_insert)} nuevos registros en res.partner.")


In [213]:

# Función para obtener los registros insertados en una ejecución específica
def get_inserted_records(execution_id):
    # Buscar la ejecución en el historial
    for execution in insertion_history:
        if execution['execution_id'] == execution_id:
            # Convertir la lista de diccionarios en un DataFrame
            df = pd.DataFrame(execution['new_records'])
            return df
    
    # Si no se encuentra la ejecución
    print(f"No se encontraron registros para la ejecución con ID: {execution_id}")
    return None

In [214]:
# Insertar nuevos registros con un ID de ejecución '1'
insert_new_partners(customers, execution_id='2')

Se insertaron 2267 nuevos registros en res.partner.


In [215]:
# Obtener los registros insertados en la ejecución '1'
#esto lo hago para tener el registro de que data acabo de importar para luego tener ese bloque de data para generar las invoices
# o incluso borrarlos cuando lleguemos a otras metodologias

df_execution_1 = get_inserted_records('2')
d_exec = pd.DataFrame(df_execution_1)
d_exec

,name
0,5acdf7d8e4b014cba3b1724b
1,5aa291c2e4b0c65fcc798ce8
2,5ad08c66e4b01bf0eefaf887
3,5cb4d5d0e4b0bd6bed7869fa
4,5b47de04e4b03c859495a0f7
...,...
2262,5a71ba83e4b07b95e31fc272
2263,5727680de4b06a6e4222f0c8
2264,53c42a63e4b08c5d83b878ff
2265,536109b2e4b0ae650c483374


In [241]:
#aqui llame al dataframe de los registros de res.partner para ver solamente el nombre y id
p_id = partner[['id', 'name']]
p_id_df = pd.DataFrame(p_id)
p_id_df

,id,name
0,86,(AR) Exento
1,87,(AR) Monotributista
2,67,0h3n-rijaune@yahoo.example.com
3,25762,16CXyqUDJkqSi1S5N
4,26495,535680f7e4b03a7c45f66703
...,...,...
2386,69,the.jose@gmail.example.com
2387,77,the.real.shane@gmail.example.com
2388,63,thing.thang.thong@gmail.example.com
2389,76,tina.turner@gmail.example.com


In [242]:
#despues uni el dataframe que me mostraba el resultado de los customers que agregue 
# para que hiciera una union con el dataframe de los partners ya que al ser exitosa la insercion de los customers en odoo
# ahora esos customers de chargeBee ya estan en Odoo
new_partners_df = pd.merge(p_id_df, d_exec, on='name', how='inner')
new_partners_df

,id,name
0,25762,16CXyqUDJkqSi1S5N
1,26495,535680f7e4b03a7c45f66703
2,26525,536109b2e4b0ae650c483374
3,26524,53c42a63e4b08c5d83b878ff
4,26521,540f3b9ee4b0492f27113fe7
...,...,...
2262,26390,66ccdda3b1c5d119cf1a2412
2263,26366,66cf6e434427b90002f38734
2264,26363,66cf6f0374ecf760b7a5b258
2265,26425,66d0a6efa9c6e736847a56e6


In [172]:
#ahora voy a eliminar esos registros ya que aun no habia corregido los customers ya que hay duplicados
partner_id_to_delete = 131  # Reemplazar con el ID real que hay que eliminar

# Realizar la operación de eliminación
odoo.execute('res.partner', 'unlink', [partner_id_to_delete])


{'jsonrpc': '2.0', 'id': 2276, 'error': {'code': 200, 'message': 'Odoo Server Error', 'data': {'name': 'odoo.exceptions.MissingError', 'debug': 'Traceback (most recent call last):\n  File "/home/odoo/src/odoo/odoo/http.py", line 1781, in _serve_db\n    return service_model.retrying(self._serve_ir_http, self.env)\n  File "/home/odoo/src/odoo/odoo/service/model.py", line 133, in retrying\n    result = func()\n  File "/home/odoo/src/odoo/odoo/http.py", line 1808, in _serve_ir_http\n    response = self.dispatcher.dispatch(rule.endpoint, args)\n  File "/home/odoo/src/odoo/odoo/http.py", line 2012, in dispatch\n    result = self.request.registry[\'ir.http\']._dispatch(endpoint)\n  File "/home/odoo/src/odoo/addons/website/models/ir_http.py", line 235, in _dispatch\n    response = super()._dispatch(endpoint)\n  File "/home/odoo/src/odoo/odoo/addons/base/models/ir_http.py", line 226, in _dispatch\n    result = endpoint(**request.params)\n  File "/home/odoo/src/odoo/odoo/http.py", line 757, in r

Exception: Command has failed

In [223]:
#genere este codigo para las ocasiones en las que hay que realizar la eliminacion de muchos registros sin que afecte a los que hay que 
# proteger ya que al hacer el llamado directo y luego borrar es lo que podria sabotear el trabajo debido a un error humano jaja
# Obtener todos los registros del modelo res.partner, ordenados por id de forma descendente y con los campos 'id' y 'name'
i5 = odoo.search_read('res.partner', [], ['id', 'name'], limit=None)

# Convertir los resultados a un DataFrame
partner2 = pd.DataFrame(i5)

# Ordenar el DataFrame por la columna 'id' en orden descendente y seleccionar la cantidad de los ultimos registros que hay que ver
last = partner2.sort_values(by='id', ascending=False).head(2267)

# Mostrar el DataFrame con los últimos registros mostrando 'id' y 'name'
last_rows_df = pd.DataFrame(last)
last_rows_df

,id,name
179,31060,5a0c58c9e4b031a2c96d319c
5,31059,536109b2e4b0ae650c483374
6,31058,53c42a63e4b08c5d83b878ff
27,31057,5727680de4b06a6e4222f0c8
215,31056,5a71ba83e4b07b95e31fc272
...,...,...
345,28798,5b47de04e4b03c859495a0f7
561,28797,5cb4d5d0e4b0bd6bed7869fa
266,28796,5ad08c66e4b01bf0eefaf887
241,28795,5aa291c2e4b0c65fcc798ce8


In [237]:
last_rows_df.loc[last_rows_df['id'] == 25762]

,id,name


In [224]:
dd = last_rows_df.drop_duplicates('name')
dd

,id,name
179,31060,5a0c58c9e4b031a2c96d319c
5,31059,536109b2e4b0ae650c483374
6,31058,53c42a63e4b08c5d83b878ff
27,31057,5727680de4b06a6e4222f0c8
215,31056,5a71ba83e4b07b95e31fc272
...,...,...
345,28798,5b47de04e4b03c859495a0f7
561,28797,5cb4d5d0e4b0bd6bed7869fa
266,28796,5ad08c66e4b01bf0eefaf887
241,28795,5aa291c2e4b0c65fcc798ce8


In [245]:
last_partners_df = pd.merge(p_id_df, last_rows_df, on='name', how='inner')
last_partners_df

,id_x,name,id_y
0,25762,16CXyqUDJkqSi1S5N,30296
1,26495,535680f7e4b03a7c45f66703,31029
2,26525,536109b2e4b0ae650c483374,31059
3,26524,53c42a63e4b08c5d83b878ff,31058
4,26521,540f3b9ee4b0492f27113fe7,31055
...,...,...,...
2262,26390,66ccdda3b1c5d119cf1a2412,30924
2263,26366,66cf6e434427b90002f38734,30900
2264,26363,66cf6f0374ecf760b7a5b258,30897
2265,26425,66d0a6efa9c6e736847a56e6,30959


In [248]:
renamed_last_partners_df = last_partners_df.rename(columns={'name': 'customer_id', 'id_y': 'partner_id'})
renamed_last_partners_df

,id_x,customer_id,partner_id
0,25762,16CXyqUDJkqSi1S5N,30296
1,26495,535680f7e4b03a7c45f66703,31029
2,26525,536109b2e4b0ae650c483374,31059
3,26524,53c42a63e4b08c5d83b878ff,31058
4,26521,540f3b9ee4b0492f27113fe7,31055
...,...,...,...
2262,26390,66ccdda3b1c5d119cf1a2412,30924
2263,26366,66cf6e434427b90002f38734,30900
2264,26363,66cf6f0374ecf760b7a5b258,30897
2265,26425,66d0a6efa9c6e736847a56e6,30959


In [250]:
partners_Odoo_invoices = pd.merge(renamed_last_partners_df, order, on='customer_id', how='inner')
partners_Odoo_invoices

,id_x,customer_id,partner_id,id,po_number,subscription_id,recurring,status,price_type,date,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded
0,25762,16CXyqUDJkqSi1S5N,30296,187055,fertestoffline,AzZnUNUDJodaa1Uow,True,payment_due,tax_exclusive,2024-08-20 09:53:33-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,26495,535680f7e4b03a7c45f66703,31029,187881,Argentina - Transferencia Bancaria (Leadaki),AzqB5PUBczDZj76jS,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,...,NaN,line_item_discount,li_16BdL2UN1xX73CrTF,item_level_coupon,2400.0,CTASPRUEBA-100%-FOREVER,CTASPRUEBA-100%-FOREVER,NaN,NaN,NaN
2,26525,536109b2e4b0ae650c483374,31059,187913,Argentina - Transferencia Bancaria (Leadaki),16CY4nUBd1nGm7OLS,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,26524,53c42a63e4b08c5d83b878ff,31058,187912,Argentina - Transferencia Bancaria (Leadaki),AzqBxKUBeN2WG88Ao,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,26521,540f3b9ee4b0492f27113fe7,31055,187909,Argentina - Transferencia Bancaria (Leadaki),AzyhzvUBeTc508oWU,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,26390,66ccdda3b1c5d119cf1a2412,30924,187751,Argentina,16CV2TUMs0jeK371d,True,paid,tax_exclusive,2024-08-29 16:34:47-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2473,26366,66cf6e434427b90002f38734,30900,187720,Mexico,AzZnUfUMqw8ci2C2q,True,paid,tax_exclusive,2024-08-29 12:10:15-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2474,26363,66cf6f0374ecf760b7a5b258,30897,187717,Colombia,16CV2TUMqv4Dc2Ag8,True,paid,tax_exclusive,2024-08-29 12:06:00-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2475,26425,66d0a6efa9c6e736847a56e6,30959,187789,Cuenta test,16CXxzUMwehmC2IUg,True,paid,tax_exclusive,2024-08-30 11:38:38-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [272]:
p_odoo_invoices = partners_Odoo_invoices.drop_duplicates('customer_id')
p_odoo_invoices

,id_x,customer_id,partner_id,id,po_number,subscription_id,recurring,status,price_type,date,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded
0,25762,16CXyqUDJkqSi1S5N,30296,187055,fertestoffline,AzZnUNUDJodaa1Uow,True,payment_due,tax_exclusive,2024-08-20 09:53:33-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,26495,535680f7e4b03a7c45f66703,31029,187881,Argentina - Transferencia Bancaria (Leadaki),AzqB5PUBczDZj76jS,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,...,NaN,line_item_discount,li_16BdL2UN1xX73CrTF,item_level_coupon,2400.0,CTASPRUEBA-100%-FOREVER,CTASPRUEBA-100%-FOREVER,NaN,NaN,NaN
2,26525,536109b2e4b0ae650c483374,31059,187913,Argentina - Transferencia Bancaria (Leadaki),16CY4nUBd1nGm7OLS,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,26524,53c42a63e4b08c5d83b878ff,31058,187912,Argentina - Transferencia Bancaria (Leadaki),AzqBxKUBeN2WG88Ao,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,26521,540f3b9ee4b0492f27113fe7,31055,187909,Argentina - Transferencia Bancaria (Leadaki),AzyhzvUBeTc508oWU,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,26390,66ccdda3b1c5d119cf1a2412,30924,187751,Argentina,16CV2TUMs0jeK371d,True,paid,tax_exclusive,2024-08-29 16:34:47-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2473,26366,66cf6e434427b90002f38734,30900,187720,Mexico,AzZnUfUMqw8ci2C2q,True,paid,tax_exclusive,2024-08-29 12:10:15-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2474,26363,66cf6f0374ecf760b7a5b258,30897,187717,Colombia,16CV2TUMqv4Dc2Ag8,True,paid,tax_exclusive,2024-08-29 12:06:00-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2475,26425,66d0a6efa9c6e736847a56e6,30959,187789,Cuenta test,16CXxzUMwehmC2IUg,True,paid,tax_exclusive,2024-08-30 11:38:38-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [271]:
order

,id,po_number,customer_id,subscription_id,recurring,status,price_type,date,due_date,net_term_days,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded
0,185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,tax_exclusive,2024-08-01 05:08:00-03:00,2024-08-01 05:08:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,tax_exclusive,2024-08-01 07:17:00-03:00,2024-08-01 07:17:00-03:00,0,...,NaN,line_item_discount,li_Azz6aoUKC1bN81M2q,item_level_coupon,2000.0,Migra2024Premium2Users,Migra2024Premium2Users,NaN,NaN,NaN
2,185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,True,paid,tax_exclusive,2024-08-01 07:39:00-03:00,2024-08-01 07:39:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,185395,Argentina - Transferencia Bancaria,5cb4d5d0e4b0bd6bed7869fa,16CY2RUEdBvEE1YCI,True,paid,tax_exclusive,2024-08-01 07:54:00-03:00,2024-08-01 07:54:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,185396,Chile,5b47de04e4b03c859495a0f7,Izy9TIBR04obyPV06,True,paid,tax_exclusive,2024-08-01 08:05:08-03:00,2024-08-01 08:05:08-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,187910,Argentina - Transferencia Bancaria (Leadaki),5a71ba83e4b07b95e31fc272,AzqBxKUBeW38u8EGG,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2467,187911,Argentina - Transferencia Bancaria (Leadaki),5727680de4b06a6e4222f0c8,AzyhzvUBeSCea8nZZ,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2468,187912,Argentina - Transferencia Bancaria (Leadaki),53c42a63e4b08c5d83b878ff,AzqBxKUBeN2WG88Ao,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2460,187913,Argentina - Transferencia Bancaria (Leadaki),536109b2e4b0ae650c483374,16CY4nUBd1nGm7OLS,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [211]:
# Obtener los IDs de los últimos  registros
partner_ids_to_delete = last['id'].tolist()

# Eliminar los registros utilizando el método 'unlink' de Odoo
if partner_ids_to_delete:
    odoo.execute('res.partner', 'unlink', [partner_ids_to_delete])

# Mostrar el número de registros eliminados
print(f"Se han eliminado {len(partner_ids_to_delete)} registros de res.partner.")

Se han eliminado 2267 registros de res.partner.


In [49]:
#aqui lo reescribo el codigo de insercion para que se vea que funciona

#aqui el codigo que genera una invoice de clientes 
odoo.create('account.move', [{
    'move_type': 'out_invoice',  # Factura de cliente
    'partner_id': 138,  # ID del cliente
    'company_id': 1,  # ID de la compañía
    'website_id': 1,  # ID del sitio web
    'campaign_id': False,
    'source_id': False,
    'medium_id': False,
    'activity_ids': [],
    'amount_total': 28.75,
    'amount_residual': 28.75,
    'amount_total_signed': 28.75,
    'amount_total_in_currency_signed': 28.75,
    'amount_residual_signed': 28.75,
    'activity_state': True,
}])

166

In [252]:
cur = odoo.search_read('account.move', [], ['currency_id'], limit=None)

# Convertir los resultados a un DataFrame
currency_c = pd.DataFrame(cur)
currency_c

,id,currency_id
0,40542,"[1, USD]"
1,40541,"[1, USD]"
2,40540,"[1, USD]"
3,40539,"[1, USD]"
4,40538,"[1, USD]"
...,...,...
2299,19,"[1, USD]"
2300,18,"[1, USD]"
2301,17,"[1, USD]"
2302,11,"[1, USD]"


In [253]:
# Revisar la estructura de los datos
print(currency_c['currency_id'].head(10))  # Revisar si los valores están en el formato correcto

0    [1, USD]
1    [1, USD]
2    [1, USD]
3    [1, USD]
4    [1, USD]
5    [1, USD]
6    [1, USD]
7    [1, USD]
8    [1, USD]
9    [1, USD]
Name: currency_id, dtype: object


In [254]:
# Asegurarse de que currency_id sean listas y cada lista tenga dos elementos
currency_c['currency_id'] = currency_c['currency_id'].apply(lambda x: x if isinstance(x, list) and len(x) == 2 else [None, None])

# Separar la columna currency_id en dos columnas: currency_num y currency_name
currency_c[['currency_num', 'currency_code']] = pd.DataFrame(currency_c['currency_id'].tolist(), index=currency_c.index)

# Convertir el campo currency_num a tipo numérico
currency_c['currency_num'] = pd.to_numeric(currency_c['currency_num'], errors='coerce')

# Verificar el resultado refinado con las nuevas columnas
currency_refined = currency_c[['currency_num', 'currency_code']]
currency_refined

,currency_num,currency_code
0,1,USD
1,1,USD
2,1,USD
3,1,USD
4,1,USD
...,...,...
2299,1,USD
2300,1,USD
2301,1,USD
2302,1,USD


In [255]:
currency_re_defined = currency_refined.drop_duplicates('currency_code')
currency_re_defined

,currency_num,currency_code
0,1,USD


In [275]:
partners_to_odoo_invoices = pd.merge(currency_re_defined, p_odoo_invoices, on='currency_code', how='inner')
partners_to_odoo_invoices

,currency_num,currency_code,id_x,customer_id,partner_id,id,po_number,subscription_id,recurring,status,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded
0,1,USD,25762,16CXyqUDJkqSi1S5N,30296,187055,fertestoffline,AzZnUNUDJodaa1Uow,True,payment_due,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,USD,26495,535680f7e4b03a7c45f66703,31029,187881,Argentina - Transferencia Bancaria (Leadaki),AzqB5PUBczDZj76jS,True,paid,...,NaN,line_item_discount,li_16BdL2UN1xX73CrTF,item_level_coupon,2400.0,CTASPRUEBA-100%-FOREVER,CTASPRUEBA-100%-FOREVER,NaN,NaN,NaN
2,1,USD,26525,536109b2e4b0ae650c483374,31059,187913,Argentina - Transferencia Bancaria (Leadaki),16CY4nUBd1nGm7OLS,True,paid,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,USD,26524,53c42a63e4b08c5d83b878ff,31058,187912,Argentina - Transferencia Bancaria (Leadaki),AzqBxKUBeN2WG88Ao,True,paid,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,USD,26521,540f3b9ee4b0492f27113fe7,31055,187909,Argentina - Transferencia Bancaria (Leadaki),AzyhzvUBeTc508oWU,True,paid,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260,1,USD,26390,66ccdda3b1c5d119cf1a2412,30924,187751,Argentina,16CV2TUMs0jeK371d,True,paid,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2261,1,USD,26366,66cf6e434427b90002f38734,30900,187720,Mexico,AzZnUfUMqw8ci2C2q,True,paid,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2262,1,USD,26363,66cf6f0374ecf760b7a5b258,30897,187717,Colombia,16CV2TUMqv4Dc2Ag8,True,paid,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2263,1,USD,26425,66d0a6efa9c6e736847a56e6,30959,187789,Cuenta test,16CXxzUMwehmC2IUg,True,paid,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
#una vez teniendo el modelo de res.partner e identificado los datos que debo insertar, generare la insercion de invoices en Odoo
#solamente como prueba ya que unicamente voy a incluir unos campos a las facturas pero lo mas importante en este caso es ingresar el
# partner_id de cada partner que ya enliste despues de ingresar el monton de registros de Chargebee en Odoo y asi de una vez saber a que
#partners debo ingresar en las invoices 
# Función para insertar facturas en account.move con validación de datos
def insert_invoices_to_account_move(partners_Odoo_invoices, execution_id):
    # Crear una lista para los nuevos registros a insertar
    records_to_insert = []
    
    # Iterar sobre el DataFrame con el id y name del partner
    for _, row in partners_Odoo_invoices.iterrows():
        # Crear el diccionario para cada factura con el partner_id
        record = {
            'move_type': 'out_invoice',  # Tipo de movimiento: Factura de cliente
            'partner_id': row['id_x'],     # Usar el ID del cliente obtenido del DataFrame
            'journal_id': 1,  # Reemplaza con el ID del diario correcto
            'currency_id': 1  # Reemplaza con el ID de la moneda correcta
            #'invoice_line_ids': [(0, 0, {
             #   'product_id': 1,  # ID de un producto de ejemplo
              #  'quantity': 1,  # Cantidad de productos
               # 'price_unit': 100,  # Precio unitario de ejemplo
                #'name': 'Example product',  # Nombre del producto
            #})],  # Detalles de la línea de factura
        }

        # Validar que el diccionario tenga los campos correctos
        if 'partner_id' not in record or not record['partner_id']:
            print(f"Registro inválido: {record}")
            continue  # Omitir registros inválidos

        records_to_insert.append(record)
    
    # Asegurarse de que la lista no esté vacía antes de hacer la inserción
    if records_to_insert:
        try:
            # Aquí se hace la inserción en Odoo
            for record in records_to_insert:
                # Enviar un solo diccionario a la vez
                result = odoo.create('account.move', [record])
            
            # Guardar los nuevos registros en el historial con un ID de ejecución
            insertion_history.append({
                'execution_id': execution_id,
                'new_records': records_to_insert
            })
            
            # Mostrar cuántos registros nuevos fueron insertados
            print(f"Se insertaron {len(records_to_insert)} nuevas facturas en account.move.")
        except Exception as e:
            print(f"Error durante la inserción: {str(e)}")
    else:
        print("No se encontraron registros válidos para insertar en account.move.")


In [60]:
# Ejemplo de uso:
insert_invoices_to_account_move(last_partners_df, execution_id='3')


Se insertaron 2272 nuevas facturas en account.move.


In [238]:
partners_to_odoo_invoices.loc[partners_to_odoo_invoices['id_y'] == 25762]
#25762

,currency_num,currency_code,id_x,customer_id,id_y,id,po_number,subscription_id,recurring,status,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded


In [276]:
#una vez teniendo el modelo de res.partner e identificado los datos que debo insertar, generare la insercion de invoices en Odoo
#solamente como prueba ya que unicamente voy a incluir unos campos a las facturas pero lo mas importante en este caso es ingresar el
# partner_id de cada partner que ya enliste despues de ingresar el monton de registros de Chargebee en Odoo y asi de una vez saber a que
#partners debo ingresar en las invoices 
# Función para insertar facturas en account.move con validación de datos
import signal
import time

# Variable global para indicar si hubo una interrupción manual o del sistema
interrupted = False

# Función para manejar señales de interrupción (como CTRL+C o kill)
def handle_interrupt(signum, frame):
    global interrupted
    print(f"Señal {signum} recibida. Iniciando rollback...")
    interrupted = True  # Marcar interrupción
    raise Exception("Proceso interrumpido manualmente o por el sistema.")

# Conectar las señales de interrupción al manejador de interrupciones
signal.signal(signal.SIGINT, handle_interrupt)   # CTRL+C
signal.signal(signal.SIGTERM, handle_interrupt)  # kill command

def insert_invoices_with_rollback(partners_to_odoo_invoices, execution_id):
    global interrupted
    records_to_insert = []
    inserted_record_ids = []

    # Iterar sobre el DataFrame
    for _, row in partners_to_odoo_invoices.iterrows():
        record = {
            'move_type': 'out_invoice',
            'partner_id': row['partner_id'],
            'journal_id': 1,
            'currency_id': row['currency_num']
        }

        if 'partner_id' not in record or not record['partner_id']:
            print(f"Registro inválido: {record}")
            continue

        records_to_insert.append(record)

    if records_to_insert:
        try:
            # Realizar inserciones en un bucle
            for record in records_to_insert:
                if interrupted:
                    raise Exception("Interrupción detectada antes de insertar.")
                
                # Crear un registro (necesitamos envolver el diccionario en una lista)
                result = odoo.execute('account.move', 'create', [record])
                inserted_record_ids.append(result)

            insertion_history.append({
                'execution_id': execution_id,
                'new_records': inserted_record_ids
            })

            print(f"Se insertaron {len(inserted_record_ids)} nuevas facturas en account.move.")

        except Exception as e:
            print(f"Error durante la inserción: {str(e)}")

            # Intentar eliminar los registros ya insertados en caso de error
            if inserted_record_ids:
                try:
                    print(f"Intentando eliminar los registros insertados debido al error...")
                    odoo.execute('account.move', 'unlink', inserted_record_ids)
                    print("Registros eliminados correctamente.")
                except Exception as rollback_error:
                    print(f"Error al intentar eliminar registros: {str(rollback_error)}")

    else:
        print("No se encontraron registros válidos para insertar en account.move.")


In [277]:
# Prueba de la función de inserción
insert_invoices_with_rollback(partners_to_odoo_invoices, execution_id='2')


Se insertaron 2265 nuevas facturas en account.move.


In [278]:
i6 = odoo.search_read('account.move', [], ['id', 'partner_id', 'currency_id'], limit=None)

# Convertir los resultados a un DataFrame
inv_df = pd.DataFrame(i6)

# Ordenar el DataFrame por la columna 'id' en orden descendente y seleccionar la cantidad de los ultimos registros que hay que ver
last_inv = inv_df.sort_values(by='id', ascending=False).head(2265)

# Mostrar el DataFrame con los últimos registros mostrando 'id' y 'name'
last_invoices = pd.DataFrame(last_inv)
last_invoices

,id,partner_id,currency_id
0,42985,"[30962, 66d1e1c49112e41a78b90d4c]","[1, USD]"
1,42984,"[30959, 66d0a6efa9c6e736847a56e6]","[1, USD]"
2,42983,"[30897, 66cf6f0374ecf760b7a5b258]","[1, USD]"
3,42982,"[30900, 66cf6e434427b90002f38734]","[1, USD]"
4,42981,"[30924, 66ccdda3b1c5d119cf1a2412]","[1, USD]"
...,...,...,...
2260,40725,"[31055, 540f3b9ee4b0492f27113fe7]","[1, USD]"
2261,40724,"[31058, 53c42a63e4b08c5d83b878ff]","[1, USD]"
2262,40723,"[31059, 536109b2e4b0ae650c483374]","[1, USD]"
2263,40722,"[31029, 535680f7e4b03a7c45f66703]","[1, USD]"


In [279]:
# Separar la columna move_id en dos columnas: move_id (numérico) y description
last_invoices[['PartnerID', 'name']] = pd.DataFrame(last_invoices['partner_id'].tolist(), index=last_invoices.index)

# Convertir el campo move_id_num a tipo numérico
last_invoices['PartnerID'] = pd.to_numeric(last_invoices['PartnerID'], errors='coerce')

# Verificar el resultado
partner_invoices = last_invoices[['id', 'PartnerID', 'name']]
partner_invoices

,id,PartnerID,name
0,42985,30962,66d1e1c49112e41a78b90d4c
1,42984,30959,66d0a6efa9c6e736847a56e6
2,42983,30897,66cf6f0374ecf760b7a5b258
3,42982,30900,66cf6e434427b90002f38734
4,42981,30924,66ccdda3b1c5d119cf1a2412
...,...,...,...
2260,40725,31055,540f3b9ee4b0492f27113fe7
2261,40724,31058,53c42a63e4b08c5d83b878ff
2262,40723,31059,536109b2e4b0ae650c483374
2263,40722,31029,535680f7e4b03a7c45f66703


In [280]:
last_moves = last_invoices.rename(columns={'id': 'move_id_num'})
last_moves

,move_id_num,partner_id,currency_id,PartnerID,name
0,42985,"[30962, 66d1e1c49112e41a78b90d4c]","[1, USD]",30962,66d1e1c49112e41a78b90d4c
1,42984,"[30959, 66d0a6efa9c6e736847a56e6]","[1, USD]",30959,66d0a6efa9c6e736847a56e6
2,42983,"[30897, 66cf6f0374ecf760b7a5b258]","[1, USD]",30897,66cf6f0374ecf760b7a5b258
3,42982,"[30900, 66cf6e434427b90002f38734]","[1, USD]",30900,66cf6e434427b90002f38734
4,42981,"[30924, 66ccdda3b1c5d119cf1a2412]","[1, USD]",30924,66ccdda3b1c5d119cf1a2412
...,...,...,...,...,...
2260,40725,"[31055, 540f3b9ee4b0492f27113fe7]","[1, USD]",31055,540f3b9ee4b0492f27113fe7
2261,40724,"[31058, 53c42a63e4b08c5d83b878ff]","[1, USD]",31058,53c42a63e4b08c5d83b878ff
2262,40723,"[31059, 536109b2e4b0ae650c483374]","[1, USD]",31059,536109b2e4b0ae650c483374
2263,40722,"[31029, 535680f7e4b03a7c45f66703]","[1, USD]",31029,535680f7e4b03a7c45f66703


In [262]:
last_moves.loc[last_moves['move_id_num'] == 20097]

,move_id_num,partner_id,currency_id,PartnerID,name


In [263]:
print_column_types(last_invoices)

id 	 int64
partner_id 	 object
currency_id 	 object
PartnerID 	 int64
name 	 object


In [270]:
# Obtener los IDs de los últimos  registros
ids_to_delete = last_inv['id'].tolist()

# Eliminar los registros utilizando el método 'unlink' de Odoo
if ids_to_delete:
    odoo.execute('account.move', 'unlink', [ids_to_delete])

# Mostrar el número de registros eliminados
print(f"Se han eliminado {len(ids_to_delete)} registros de account.move.")

Se han eliminado 178 registros de account.move.


In [52]:
order

,id,po_number,customer_id,subscription_id,recurring,status,price_type,date,due_date,net_term_days,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded
0,185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,tax_exclusive,2024-08-01 05:08:00-03:00,2024-08-01 05:08:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,tax_exclusive,2024-08-01 07:17:00-03:00,2024-08-01 07:17:00-03:00,0,...,NaN,line_item_discount,li_Azz6aoUKC1bN81M2q,item_level_coupon,2000.0,Migra2024Premium2Users,Migra2024Premium2Users,NaN,NaN,NaN
2,185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,True,paid,tax_exclusive,2024-08-01 07:39:00-03:00,2024-08-01 07:39:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,185395,Argentina - Transferencia Bancaria,5cb4d5d0e4b0bd6bed7869fa,16CY2RUEdBvEE1YCI,True,paid,tax_exclusive,2024-08-01 07:54:00-03:00,2024-08-01 07:54:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,185396,Chile,5b47de04e4b03c859495a0f7,Izy9TIBR04obyPV06,True,paid,tax_exclusive,2024-08-01 08:05:08-03:00,2024-08-01 08:05:08-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,187910,Argentina - Transferencia Bancaria (Leadaki),5a71ba83e4b07b95e31fc272,AzqBxKUBeW38u8EGG,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2467,187911,Argentina - Transferencia Bancaria (Leadaki),5727680de4b06a6e4222f0c8,AzyhzvUBeSCea8nZZ,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2468,187912,Argentina - Transferencia Bancaria (Leadaki),53c42a63e4b08c5d83b878ff,AzqBxKUBeN2WG88Ao,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2460,187913,Argentina - Transferencia Bancaria (Leadaki),536109b2e4b0ae650c483374,16CY4nUBd1nGm7OLS,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
Chargebee_li = order[['id_line_items_expanded', 'date_from', 'date_to', 'unit_amount', 'quantity', 'amount', 'pricing_model', 'is_taxed',
       'tax_amount', 'object_line_items_expanded', 'description', 'entity_type', 'entity_id', 'entity_description', 'tax_exempt_reason'
       , 'discount_amount', 'item_level_discount_amount', 'starting_unit', 'ending_unit', 'quantity_used',
       'unit_amount_line_item_tiers_expanded', 'line_item_id', 'txn_id', 'applied_amount', 'applied_at', 'txn_status', 'txn_date',
       'txn_amount', 'applied_amount_applied_credits_expanded', 'applied_at_applied_credits_expanded', 'cn_id', 'cn_reason_code',
       'cn_create_reason_code', 'cn_date', 'cn_status', 'cn_id_adjustment_credit_notes_expanded',
       'cn_reason_code_adjustment_credit_notes_expanded', 'cn_create_reason_code_adjustment_credit_notes_expanded',
       'cn_date_adjustment_credit_notes_expanded', 'cn_total', 'cn_status_adjustment_credit_notes_expanded',
       'cn_id_issued_credit_notes_expanded', 'cn_reason_code_issued_credit_notes_expanded',
       'cn_create_reason_code_issued_credit_notes_expanded', 'cn_date_issued_credit_notes_expanded',
       'cn_total_issued_credit_notes_expanded', 'cn_status_issued_credit_notes_expanded', 'created_at', 'attempt', 'dunning_type',
       'transaction_id', 'txn_status_dunning_attempts_expanded', 'txn_amount_dunning_attempts_expanded', 'object_discounts_expanded',
       'entity_type_discounts_expanded', 'description_discounts_expanded', 'amount_discounts_expanded', 'discount_type',
       'discount_percentage', 'object_line_item_discounts_expanded', 'line_item_id_line_item_discounts_expanded',
       'discount_type_line_item_discounts_expanded', 'discount_amount_line_item_discounts_expanded', 'coupon_id',
       'entity_id_line_item_discounts_expanded', 'note', 'entity_type_notes_expanded', 'entity_type_notes_expanded']]

In [281]:
cbli = Chargebee_li.drop_duplicates('id_line_items_expanded')
cbli

,id_line_items_expanded,date_from,date_to,unit_amount,quantity,amount,pricing_model,is_taxed,tax_amount,object_line_items_expanded,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_type_notes_expanded
0,li_6olL2UKBV7cg22wd,2024-08-01 05:08:00-03:00,2024-09-01 05:08:00-03:00,8900,1,8900,flat_fee,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,li_6olL2UKBV7cm22we,2024-08-01 05:08:00-03:00,2024-09-01 05:08:00-03:00,0,1,0,tiered,False,0,line_item,...,NaN,line_item_discount,li_Azz6aoUKC1bN81M2q,item_level_coupon,2000.0,Migra2024Premium2Users,Migra2024Premium2Users,NaN,NaN,NaN
2,li_Azz6aoUKC1bN11M2p,2024-08-01 07:17:00-03:00,2024-09-01 07:17:00-03:00,8900,1,8900,flat_fee,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,li_Azz6aoUKC1bN81M2q,2024-08-01 07:17:00-03:00,2024-09-01 07:17:00-03:00,750,4,3000,tiered,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,li_AzyjxKUKC78gO1LOQ,2024-08-01 07:39:00-03:00,2024-09-01 07:39:00-03:00,39,100,3900,stairstep,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,li_AzZnNZULMe4v73Smk,2024-08-13 17:21:28-03:00,2024-09-13 17:21:28-03:00,5900,1,5900,per_unit,False,0,line_item,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2467,li_16BdPRULMXR8o76Yh,2024-08-13 16:55:04-03:00,2024-09-13 16:55:04-03:00,0,3,0,tiered,False,0,line_item,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2468,li_6ol3xULMdysQ59qN,2024-08-13 17:21:05-03:00,2024-09-10 15:22:14-03:00,901,1,901,tiered,False,0,line_item,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2460,li_16BdPRULMQXnv6u2P,2024-08-13 16:27:41-03:00,2024-09-13 16:27:41-03:00,39,100,3900,stairstep,False,0,line_item,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [282]:
import signal

# Variable global para indicar si hubo una interrupción manual o del sistema
interrupted = False

# Función para manejar señales de interrupción (como CTRL+C o kill)
def handle_interrupt(signum, frame):
    global interrupted
    print(f"Señal {signum} recibida. Iniciando rollback...")
    interrupted = True  # Marcar interrupción
    raise Exception("Proceso interrumpido manualmente o por el sistema.")

# Conectar las señales de interrupción al manejador de interrupciones
signal.signal(signal.SIGINT, handle_interrupt)   # CTRL+C
signal.signal(signal.SIGTERM, handle_interrupt)  # kill command

def insert_account_move_lines_with_rollback(partner_invoices, execution_id):
    global interrupted
    records_to_insert = []
    inserted_record_ids = []
    unique_combinations = set()  # Para almacenar combinaciones únicas de move_id y partner_id

    # Eliminar duplicados en el DataFrame antes de procesar
    partner_invoices = partner_invoices.drop_duplicates(subset=['id', 'PartnerID'])

    # Iterar sobre las filas del DataFrame sin duplicar
    for _, row in partner_invoices.iterrows():
        move_id = row['id']
        partner_id = row['PartnerID']

        # Evitar duplicados verificando la combinación única de move_id y partner_id
        if (move_id, partner_id) in unique_combinations:
            print(f"Combinación duplicada detectada: move_id {move_id} y partner_id {partner_id}. Saltando.")
            continue  # Saltar duplicados

        unique_combinations.add((move_id, partner_id))  # Añadir la combinación a la lista de únicas

        record = {
            'move_id': move_id,
            'name': 'Venta de producto',  # Descripción de la línea
            'quantity': 1.0,  # Cantidad
            'price_unit': 28.75,  # Precio unitario
            'debit': 0.0,  # Débito
            'partner_id': partner_id,  # ID del cliente
            'currency_id': 1,  # ID de la moneda
        }

        # Verificar si 'move_id' es válido
        if 'move_id' not in record or not record['move_id']:
            print(f"Registro inválido: {record}")
            continue

        records_to_insert.append(record)

    if records_to_insert:
        try:
            # Realizar inserciones en un bucle
            for record in records_to_insert:
                if interrupted:
                    raise Exception("Interrupción detectada antes de insertar.")
                
                # Crear un registro (necesitamos envolver el diccionario en una lista)
                result = odoo.execute('account.move.line', 'create', [record])
                inserted_record_ids.append(result)

            print(f"Se insertaron {len(inserted_record_ids)} nuevas líneas en account.move.line.")

        except Exception as e:
            print(f"Error durante la inserción: {str(e)}")

            # Intentar eliminar los registros ya insertados en caso de error
            if inserted_record_ids:
                try:
                    print(f"Intentando eliminar los registros insertados debido al error...")
                    odoo.execute('account.move.line', 'unlink', inserted_record_ids)
                    print("Registros eliminados correctamente.")
                except Exception as rollback_error:
                    print(f"Error al intentar eliminar registros: {str(rollback_error)}")

    else:
        print("No se encontraron registros válidos para insertar en account.move.line.")

In [283]:
# Llamar la función con el DataFrame partner_invoices
insert_account_move_lines_with_rollback(partner_invoices, execution_id='1')

Se insertaron 2265 nuevas líneas en account.move.line.


In [287]:
i7 = odoo.search_read('account.move.line', [], ['id', 'move_id'], limit=None)

# Convertir los resultados a un DataFrame
df_line_items = pd.DataFrame(i7)

# Ordenar el DataFrame por la columna 'id' en orden descendente y seleccionar la cantidad de los ultimos registros que hay que ver
last_li_inv = df_line_items.sort_values(by='id', ascending=False).head(4530)

# Mostrar el DataFrame con los últimos registros mostrando 'id' y 'name'
last_invoices_lines = pd.DataFrame(last_li_inv)
last_invoices_lines

,id,move_id
4529,49465,"[40721, Draft Invoice (* 40721)]"
4528,49464,"[40721, Draft Invoice (* 40721)]"
4527,49463,"[40722, Draft Invoice (* 40722)]"
4526,49462,"[40722, Draft Invoice (* 40722)]"
4525,49461,"[40723, Draft Invoice (* 40723)]"
...,...,...
4,44940,"[42983, Draft Invoice (* 42983)]"
3,44939,"[42984, Draft Invoice (* 42984)]"
2,44938,"[42984, Draft Invoice (* 42984)]"
1,44937,"[42985, Draft Invoice (* 42985)]"


In [62]:
last_invoices_lines.loc[last_invoices_lines['id'] == 28629]

,id,move_id


In [63]:
methods = odoo.execute('account.move', 'fields_get', [])
print(methods)


{'campaign_id': {'change_default': False, 'company_dependent': False, 'context': {}, 'default_export_compatible': False, 'depends': [], 'domain': [], 'exportable': True, 'help': 'This is a name that helps you keep track of your different campaign efforts, e.g. Fall_Drive, Christmas_Special', 'manual': False, 'name': 'campaign_id', 'readonly': False, 'relation': 'utm.campaign', 'required': False, 'searchable': True, 'sortable': True, 'store': True, 'string': 'Campaign', 'type': 'many2one'}, 'source_id': {'change_default': False, 'company_dependent': False, 'context': {}, 'default_export_compatible': False, 'depends': [], 'domain': [], 'exportable': True, 'help': 'This is the source of the link, e.g. Search Engine, another domain, or name of email list', 'manual': False, 'name': 'source_id', 'readonly': False, 'relation': 'utm.source', 'required': False, 'searchable': True, 'sortable': True, 'store': True, 'string': 'Source', 'type': 'many2one'}, 'medium_id': {'change_default': False, 'c

In [64]:
methods = odoo.execute('account.move', 'fields_get', [])
relevant_methods = [method for method in methods if 'cancel' in method or 'reverse' in method or 'unlink' in method]
print(relevant_methods)


['reversed_entry_id', 'need_cancel_request', 'edi_show_cancel_button', 'edi_show_abandon_cancel_button', 'edi_show_force_cancel_button', 'allow_move_with_valuation_cancelation', 'l10n_ar_afip_fce_is_cancellation']


In [65]:
# Ver los primeros 10 valores de la columna move_id para analizar su formato
print(last_invoices_lines['move_id'].head(10))

# Ver el tipo de dato de la columna move_id
print(last_invoices_lines['move_id'].apply(type).head(10))


4545    [33530, Draft Invoice (* 33530)]
4544    [33530, Draft Invoice (* 33530)]
4543    [33531, Draft Invoice (* 33531)]
4542    [33531, Draft Invoice (* 33531)]
4541    [33532, Draft Invoice (* 33532)]
4540    [33532, Draft Invoice (* 33532)]
4539    [33533, Draft Invoice (* 33533)]
4538    [33533, Draft Invoice (* 33533)]
4537    [33534, Draft Invoice (* 33534)]
4536    [33534, Draft Invoice (* 33534)]
Name: move_id, dtype: object
4545    <class 'list'>
4544    <class 'list'>
4543    <class 'list'>
4542    <class 'list'>
4541    <class 'list'>
4540    <class 'list'>
4539    <class 'list'>
4538    <class 'list'>
4537    <class 'list'>
4536    <class 'list'>
Name: move_id, dtype: object


In [66]:
# Imprimir los primeros valores del DataFrame completo
print(last_invoices_lines.head(10))


         id                           move_id
4545  44935  [33530, Draft Invoice (* 33530)]
4544  44934  [33530, Draft Invoice (* 33530)]
4543  44933  [33531, Draft Invoice (* 33531)]
4542  44932  [33531, Draft Invoice (* 33531)]
4541  44931  [33532, Draft Invoice (* 33532)]
4540  44930  [33532, Draft Invoice (* 33532)]
4539  44929  [33533, Draft Invoice (* 33533)]
4538  44928  [33533, Draft Invoice (* 33533)]
4537  44927  [33534, Draft Invoice (* 33534)]
4536  44926  [33534, Draft Invoice (* 33534)]


In [92]:
# Separar la columna move_id en dos columnas: move_id (numérico) y description
last_invoices_lines[['move_id_num', 'description']] = pd.DataFrame(last_invoices_lines['move_id'].tolist(), index=last_invoices_lines.index)

# Convertir el campo move_id_num a tipo numérico
last_invoices_lines['move_id_num'] = pd.to_numeric(last_invoices_lines['move_id_num'], errors='coerce')

# Verificar el resultado
last_invoices_lines_refined = last_invoices_lines[['id', 'move_id_num', 'description']]
last_invoices_lines_refined

,id,move_id_num,description
4,480,163,Draft Invoice (* 163)
3,479,163,Draft Invoice (* 163)
7,478,162,INV/2024/00012 (123456)
6,477,162,INV/2024/00012 (123456)
5,476,162,INV/2024/00012 (123456)
...,...,...,...
54,5,1,INV/2024/00002
63,4,16,MISC/2024/07/0002
62,3,16,MISC/2024/07/0002
65,2,15,MISC/2024/07/0001


In [73]:
#last_invoices
lines_join = pd.merge(last_moves, last_invoices_lines_refined, on='move_id_num', how='inner')
lines_join

,move_id_num,partner_id,PartnerID,name,id,description
0,35802,"[26428, 66d1e1c49112e41a78b90d4c]",26428,66d1e1c49112e41a78b90d4c,40391,Draft Invoice (* 35802)
1,35802,"[26428, 66d1e1c49112e41a78b90d4c]",26428,66d1e1c49112e41a78b90d4c,40390,Draft Invoice (* 35802)
2,35801,"[26425, 66d0a6efa9c6e736847a56e6]",26425,66d0a6efa9c6e736847a56e6,40393,Draft Invoice (* 35801)
3,35801,"[26425, 66d0a6efa9c6e736847a56e6]",26425,66d0a6efa9c6e736847a56e6,40392,Draft Invoice (* 35801)
4,35800,"[26363, 66cf6f0374ecf760b7a5b258]",26363,66cf6f0374ecf760b7a5b258,40395,Draft Invoice (* 35800)
...,...,...,...,...,...,...
4541,33532,"[26525, 536109b2e4b0ae650c483374]",26525,536109b2e4b0ae650c483374,44930,Draft Invoice (* 33532)
4542,33531,"[26495, 535680f7e4b03a7c45f66703]",26495,535680f7e4b03a7c45f66703,44933,Draft Invoice (* 33531)
4543,33531,"[26495, 535680f7e4b03a7c45f66703]",26495,535680f7e4b03a7c45f66703,44932,Draft Invoice (* 33531)
4544,33530,"[25762, 16CXyqUDJkqSi1S5N]",25762,16CXyqUDJkqSi1S5N,44935,Draft Invoice (* 33530)


In [70]:
lines_join.loc[lines_join['move_id_num']==20152]

,move_id_num,partner_id,PartnerID,name,id,description


In [71]:
# Reversar los movimientos contables
move_ids = last_invoices_lines['move_id_num'].dropna().astype(int).tolist()  # Convertir a enteros y eliminar valores NaN
print("IDs de movimientos a procesar:", move_ids)


IDs de movimientos a procesar: [33530, 33530, 33531, 33531, 33532, 33532, 33533, 33533, 33534, 33534, 33535, 33535, 33536, 33536, 33537, 33537, 33538, 33538, 33539, 33539, 33540, 33540, 33541, 33541, 33542, 33542, 33543, 33543, 33544, 33544, 33545, 33545, 33546, 33546, 33547, 33547, 33548, 33548, 33549, 33549, 33550, 33550, 33551, 33551, 33552, 33552, 33553, 33553, 33554, 33554, 33555, 33555, 33556, 33556, 33557, 33557, 33558, 33558, 33559, 33559, 33560, 33560, 33561, 33561, 33562, 33562, 33563, 33563, 33564, 33564, 33565, 33565, 33566, 33566, 33567, 33567, 33568, 33568, 33569, 33569, 33570, 33570, 33571, 33571, 33572, 33572, 33573, 33573, 33574, 33574, 33575, 33575, 33576, 33576, 33577, 33577, 33578, 33578, 33579, 33579, 33580, 33580, 33581, 33581, 33582, 33582, 33583, 33583, 33584, 33584, 33585, 33585, 33586, 33586, 33587, 33587, 33588, 33588, 33589, 33589, 33590, 33590, 33591, 33591, 33592, 33592, 33593, 33593, 33594, 33594, 33595, 33595, 33596, 33596, 33597, 33597, 33598, 33598, 33

In [64]:
move_record = odoo.execute('account.move', 'search_read', [[('id', '=', 20092)]], {'fields': ['state']})
print(f"Estado del movimiento con ID 20091: {move_record}")


{'jsonrpc': '2.0', 'id': 18, 'result': []}


Exception: Command has failed

In [75]:
# Reversar los movimientos contables
move_ids = last_invoices_lines_refined['move_id_num'].dropna().astype(int).tolist()  # Convertir a enteros y eliminar valores NaN

# Usar un conjunto para asegurarse de no procesar IDs duplicados
processed_ids = set()

if move_ids:
    try:
        for move_id in move_ids:
            # Verificar si el ID ya ha sido procesado
            if move_id in processed_ids:
                print(f"ID de movimiento {move_id} ya fue procesado, saltando...")
                continue
            
            print(f"Procesando ID de movimiento: {move_id} (Tipo: {type(move_id)})")
            
            # Verificar si el movimiento existe
            move_record = odoo.execute('account.move', 'search_read', [[('id', '=', move_id)]], {'fields': ['state']})
            
            if move_record:
                if move_record[0]['state'] != 'draft':
                    # Si el estado no es 'draft', cambiarlo a 'draft'
                    odoo.execute('account.move', 'button_draft', [[move_id]])
                    print(f"Asiento contable {move_id} puesto en estado borrador.")
                
                # Eliminar el movimiento contable
                odoo.execute('account.move', 'unlink', [[move_id]])
                print(f"Asiento contable {move_id} eliminado.")
                
                # Marcar el ID como procesado
                processed_ids.add(move_id)
            else:
                # Si no encuentra el movimiento, continuar con el siguiente
                print(f"Asiento contable {move_id} no encontrado, posiblemente ya ha sido eliminado previamente.")
                processed_ids.add(move_id)  # Marcarlo como procesado para evitar reintentos
                continue  # Sigue con el siguiente movimiento

        print(f"Se han revertido y eliminado {len(processed_ids)} asientos contables.")
        
    except Exception as e:
        print(f"Error al revertir/eliminar los asientos: {str(e)}")
else:
    print("No se encontraron IDs válidos para revertir/eliminar.")


Procesando ID de movimiento: 33530 (Tipo: <class 'int'>)
Asiento contable 33530 eliminado.
ID de movimiento 33530 ya fue procesado, saltando...
Procesando ID de movimiento: 33531 (Tipo: <class 'int'>)
Asiento contable 33531 eliminado.
ID de movimiento 33531 ya fue procesado, saltando...
Procesando ID de movimiento: 33532 (Tipo: <class 'int'>)
Asiento contable 33532 eliminado.
ID de movimiento 33532 ya fue procesado, saltando...
Procesando ID de movimiento: 33533 (Tipo: <class 'int'>)
Asiento contable 33533 eliminado.
ID de movimiento 33533 ya fue procesado, saltando...
Procesando ID de movimiento: 33534 (Tipo: <class 'int'>)
Asiento contable 33534 eliminado.
ID de movimiento 33534 ya fue procesado, saltando...
Procesando ID de movimiento: 33535 (Tipo: <class 'int'>)
Asiento contable 33535 eliminado.
ID de movimiento 33535 ya fue procesado, saltando...
Procesando ID de movimiento: 33536 (Tipo: <class 'int'>)
Asiento contable 33536 eliminado.
ID de movimiento 33536 ya fue procesado, salt

In [ ]:
#habian 94 registros

In [93]:
#hacer el llamado a acoount.move.line para mas adelante poder insertar los line items
# Especificar los campos que deseas leer
fields_to_read = ['id', 'move_id', 'product_id', 'quantity', 'price_unit', 'debit', 'credit', 'partner_id',
                  ]

# Realizar el llamado al modelo account.move.line con un límite de registros
line_items = odoo.search_read('account.move.line', [], fields_to_read, limit=4640)

# Convertir los resultados a un DataFrame
li_df = pd.DataFrame(line_items)
li_df


,id,move_id,product_id,quantity,price_unit,debit,credit,partner_id
0,40390,"[35802, Draft Invoice (* 35802)]",False,1.0,28.75,0.00,28.75,"[26428, 66d1e1c49112e41a78b90d4c]"
1,40391,"[35802, Draft Invoice (* 35802)]",False,0.0,0.00,28.75,0.00,"[26428, 66d1e1c49112e41a78b90d4c]"
2,40392,"[35801, Draft Invoice (* 35801)]",False,1.0,28.75,0.00,28.75,"[26425, 66d0a6efa9c6e736847a56e6]"
3,40393,"[35801, Draft Invoice (* 35801)]",False,0.0,0.00,28.75,0.00,"[26425, 66d0a6efa9c6e736847a56e6]"
4,40394,"[35800, Draft Invoice (* 35800)]",False,1.0,28.75,0.00,28.75,"[26363, 66cf6f0374ecf760b7a5b258]"
...,...,...,...,...,...,...,...,...
4635,36,"[11, RBILL/2023/05/0001]",False,0.0,0.00,0.00,81.17,"[14, Azure Interior]"
4636,49,"[11, RBILL/2023/05/0001]",False,0.0,0.00,622.27,0.00,"[14, Azure Interior]"
4637,15,"[7, BILL/2023/05/0001 (INV/2023/0057)]",False,1.0,541.10,541.10,0.00,"[14, Azure Interior]"
4638,32,"[7, BILL/2023/05/0001 (INV/2023/0057)]",False,0.0,0.00,81.17,0.00,"[14, Azure Interior]"


In [113]:
line_id = empties = li_df.loc[li_df['id'] == 28629]
line_id

,id,move_id,product_id,quantity,price_unit,debit,credit,partner_id


In [54]:
# Obtener la descripción del modelo 'account.move.line'
fields_description = odoo.execute('account.move.line', 'fields_get', [], {'attributes': ['string', 'type']})
lines = pd.DataFrame(fields_description)
lines

,analytic_distribution,analytic_distribution_search,analytic_precision,move_id,journal_id,company_id,company_currency_id,move_name,parent_state,date,...,invoice_origin,payment_matched_amount,l10n_pe_group_id,filter_amount,tax_settlement_move_id,tax_state,subscription_id,subscription_mrr,move_line_ids,withholding_id
string,Analytic Distribution,Analytic Distribution Search,Analytic Precision,Journal Entry,Journal,Company,Company Currency,Number,Status,Date,...,Origin,Payment Matched Amount,Group,Filter Amount,Tax Settlement Move,Tax State,Subscription,Monthly Recurring Revenue,Related Stock Moves,Withholding
type,json,json,integer,many2one,many2one,many2one,many2one,char,selection,date,...,char,monetary,many2one,float,many2one,selection,many2one,monetary,many2many,many2one
searchable,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
# Mostrar las columnas (campos) del modelo
for field, description in fields_description.items():
    print(f"Campo: {field}, Tipo: {description['type']}, Descripción: {description['string']}")

Campo: analytic_distribution, Tipo: json, Descripción: Analytic Distribution
Campo: analytic_distribution_search, Tipo: json, Descripción: Analytic Distribution Search
Campo: analytic_precision, Tipo: integer, Descripción: Analytic Precision
Campo: move_id, Tipo: many2one, Descripción: Journal Entry
Campo: journal_id, Tipo: many2one, Descripción: Journal
Campo: company_id, Tipo: many2one, Descripción: Company
Campo: company_currency_id, Tipo: many2one, Descripción: Company Currency
Campo: move_name, Tipo: char, Descripción: Number
Campo: parent_state, Tipo: selection, Descripción: Status
Campo: date, Tipo: date, Descripción: Date
Campo: invoice_date, Tipo: date, Descripción: Invoice/Bill Date
Campo: ref, Tipo: char, Descripción: Reference
Campo: is_storno, Tipo: boolean, Descripción: Company Storno Accounting
Campo: sequence, Tipo: integer, Descripción: Sequence
Campo: move_type, Tipo: selection, Descripción: Type
Campo: account_id, Tipo: many2one, Descripción: Account
Campo: name, Tip

In [144]:
odoo.create('account.move.line', [{
    'move_id': 20271,  # ID del movimiento contable (relacionado con account.move)
    #'account_id': 3,  # ID de la cuenta contable
    'name': 'Venta de producto',  # Descripción de la línea
    'quantity': 1.0,  # Cantidad
    'price_unit': 28.75,  # Precio unitario
    'debit': 0.0,  # Débito
    #'credit': 28.75,  # Crédito
    'partner_id': 21032,  # ID del cliente
    #'company_id': 1,  # ID de la compañía
    'currency_id': 1,  # ID de la moneda
    #'date_maturity': '2024-09-30',  # Fecha de vencimiento
    #'analytic_account_id': False,  # ID de la cuenta analítica (si aplica)
    #'tax_ids': [(6, 0, [])],  # Impuestos aplicables (lista vacía si no hay)
}])


28270

In [151]:
#ahora voy a eliminar esos registros ya que aun no habia corregido los customers ya que hay duplicados
move_to_delete = 28623  # Reemplazar con el ID real que hay que eliminar

# Realizar la operación de eliminación
odoo.execute('account.move.line', 'unlink', [move_to_delete])

{'jsonrpc': '2.0', 'id': 28, 'error': {'code': 200, 'message': 'Odoo Server Error', 'data': {'name': 'odoo.exceptions.ValidationError', 'debug': 'Traceback (most recent call last):\n  File "/home/odoo/src/odoo/odoo/http.py", line 1781, in _serve_db\n    return service_model.retrying(self._serve_ir_http, self.env)\n  File "/home/odoo/src/odoo/odoo/service/model.py", line 133, in retrying\n    result = func()\n  File "/home/odoo/src/odoo/odoo/http.py", line 1808, in _serve_ir_http\n    response = self.dispatcher.dispatch(rule.endpoint, args)\n  File "/home/odoo/src/odoo/odoo/http.py", line 2012, in dispatch\n    result = self.request.registry[\'ir.http\']._dispatch(endpoint)\n  File "/home/odoo/src/odoo/addons/website/models/ir_http.py", line 235, in _dispatch\n    response = super()._dispatch(endpoint)\n  File "/home/odoo/src/odoo/odoo/addons/base/models/ir_http.py", line 226, in _dispatch\n    result = endpoint(**request.params)\n  File "/home/odoo/src/odoo/odoo/http.py", line 757, in 

Exception: Command has failed

In [136]:
# Búsqueda de productos en el modelo 'product.product'
product_search = odoo.search_read('product.product', [], ['id', 'name'], limit=None)

# Convertir los resultados a un DataFrame para una mejor visualización
products_df = pd.DataFrame(product_search)

# Mostrar los productos encontrados
products_df


,id,name
0,94,Ad-Valorem
1,88,Tariff
2,86,Import Clearance
3,89,Guard Service
4,87,Statistics Rate
...,...,...
104,112,Senior Architect (Invoice on Timesheets)
105,110,Service on Timesheets
106,37,Top-up eWallet
107,4,Virtual Home Staging


In [153]:
products_df.loc[products_df['id'] == 118]

,id,name
73,118,Producto de ejemplo


In [70]:
# Datos del nuevo producto a insertar
new_product_data = {
    'name': 'Producto de ejemplo',
    'type': 'consu',  # tipo de producto ('consu' = Consumible, 'service' = Servicio, 'product' = Almacenable)
    'list_price': 100.00,  # Precio de venta
    'standard_price': 70.00,  # Precio de coste
    'default_code': 'PROD_EX',  # Código interno del producto
    'sale_ok': True,  # Habilitar para ventas
    'purchase_ok': True  # Habilitar para compras
}

# Insertar el nuevo producto usando el método Odoo.create
try:
    new_product_id = odoo.create('product.product', [new_product_data])
    print(f"Producto insertado con éxito. ID: {new_product_id}")
except Exception as e:
    print(f"Error durante la inserción del producto: {str(e)}")


Producto insertado con éxito. ID: 118


In [154]:
#ahora voy a eliminar esos registros ya que aun no habia corregido los customers ya que hay duplicados
prodcut_to_delete = 118  # Reemplazar con el ID real que hay que eliminar

# Realizar la operación de eliminación
odoo.execute('product.product', 'unlink', [move_to_delete])


{'jsonrpc': '2.0', 'id': 23, 'error': {'code': 200, 'message': 'Odoo Server Error', 'data': {'name': 'odoo.exceptions.MissingError', 'debug': 'Traceback (most recent call last):\n  File "/home/odoo/src/odoo/odoo/http.py", line 1781, in _serve_db\n    return service_model.retrying(self._serve_ir_http, self.env)\n  File "/home/odoo/src/odoo/odoo/service/model.py", line 133, in retrying\n    result = func()\n  File "/home/odoo/src/odoo/odoo/http.py", line 1808, in _serve_ir_http\n    response = self.dispatcher.dispatch(rule.endpoint, args)\n  File "/home/odoo/src/odoo/odoo/http.py", line 2012, in dispatch\n    result = self.request.registry[\'ir.http\']._dispatch(endpoint)\n  File "/home/odoo/src/odoo/addons/website/models/ir_http.py", line 235, in _dispatch\n    response = super()._dispatch(endpoint)\n  File "/home/odoo/src/odoo/odoo/addons/base/models/ir_http.py", line 226, in _dispatch\n    result = endpoint(**request.params)\n  File "/home/odoo/src/odoo/odoo/http.py", line 757, in rou

Exception: Command has failed

In [120]:
# Definir el ID de producto a buscar
product_id_to_delete = 118

# Búsqueda de productos en el modelo 'stock.move'
inventory = odoo.search_read('stock.move', [], ['product_id'], limit=None)

# Convertir los resultados a un DataFrame para una mejor visualización
inventory_df = pd.DataFrame(inventory)

# Extraer el ID del campo 'product_id', que es una lista [ID, Nombre]
# Crear una nueva columna en el DataFrame que contenga solo el número de product_id
inventory_df['product_id_number'] = inventory_df['product_id'].apply(lambda x: x[0] if isinstance(x, list) else None)

# Buscar el registro que tenga el product_id igual a 118
result = inventory_df.loc[inventory_df['product_id_number'] == product_id_to_delete]

# Mostrar los resultados encontrados
if not result.empty:
    print(f"Se encontró el producto con ID {product_id_to_delete} en 'el modelo':\n")
    print(result)
else:
    print(f"No se encontró el producto con ID {product_id_to_delete} en 'el modelo'.")


No se encontró el producto con ID 118 en 'el modelo'.


In [66]:
#hacer el llamado a res.users para mas adelante poder insertar los line items
# Especificar los campos que deseas leer
fields_to_read = ['id', 'name']

# Realizar el llamado al modelo account.move.line con un límite de registros
users = odoo.search_read('res.users', [], fields_to_read, limit=1000)

# Convertir los resultados a un DataFrame
u_df = pd.DataFrame(users)
u_df

,id,name
0,15,Chargebee
1,16,Jim Galvez
2,7,Joel Willis
3,6,Marc Demo
4,10,Valentino
5,11,Valentino
6,14,Valentino
7,13,Valentino
8,12,Valentino


In [95]:
# Agrupar por 'customer_id' y contar la cantidad de veces que aparece
grouped_df = order.groupby('customer_id').size().reset_index(name='cantidad')

# Filtrar para mostrar solo aquellos donde 'cantidad' sea mayor a 1
filtered_df = grouped_df[grouped_df['cantidad'] > 1]

# Mostrar el resultado
filtered_df

,customer_id,cantidad
6,54593f06e4b0d1a073c7326a,2
37,57bddbafe4b08f5c877fe039,16
90,58e53fbee4b03d5864b5f8ed,3
160,59d406fce4b06559f89c90a5,2
169,59f10b2ee4b05c78cc0f30a0,2
...,...,...
2236,66a9730ebf9235539751ef64,2
2240,66abf6da6fdff920a44d0034,2
2253,66b63424025ef045ba3e8958,2
2254,66ba1a84d427e13d318e70a1,2


In [ ]:
# Sumar la columna 'cantidad' del DataFrame filtrado
total_cantidad = filtered_df['cantidad'].sum()

# Mostrar la sumatoria
print(f"La sumatoria total de la columna 'cantidad' es: {total_cantidad}")


In [293]:
cbli.select_dtypes(include=['object'])

,id_line_items_expanded,pricing_model,object_line_items_expanded,description,entity_type,entity_id,entity_description,tax_exempt_reason,line_item_id,txn_id,...,description_discounts_expanded,discount_type,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_type_notes_expanded
0,li_6olL2UKBV7cg22wd,flat_fee,line_item,Cliengo Premium,plan,CLIENGO_PREMIUM_CONVER,"Cliengo Premium, 1000 conversaciones al mes",tax_not_configured,li_6olL2UKBV7cm22we,txn_AzqB1XUMIWXep30Ov,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,li_6olL2UKBV7cm22we,tiered,line_item,Agentes,addon,CLIENGO_PREMIUM_CONVER_USERS,NaN,tax_not_configured,li_Azz6aoUKC1bN81M2q,txn_16CXuLULLoKeC2h8c,...,Bonificación usuarios,fixed_amount,line_item_discount,li_Azz6aoUKC1bN81M2q,item_level_coupon,Migra2024Premium2Users,Migra2024Premium2Users,NaN,NaN,NaN
2,li_Azz6aoUKC1bN11M2p,flat_fee,line_item,Cliengo Premium,plan,CLIENGO_PREMIUM_CONVER,"Cliengo Premium, 1000 conversaciones al mes",tax_not_configured,li_Azz6aoUKC1bN81M2q,txn_16BW6dUJLpJrc1Cpj,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,li_Azz6aoUKC1bN81M2q,tiered,line_item,Agentes,addon,CLIENGO_PREMIUM_CONVER_USERS,NaN,tax_not_configured,li_AzyjxKUKC78gU1LOR,txn_AzqBCbUKy7RvD2RIN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,li_AzyjxKUKC78gO1LOQ,stairstep,line_item,Cliengo Starter,plan,CLIENGO_STARTER_50_V2,NaN,tax_not_configured,li_6olL2UKCAuit39NF,txn_6olL2UKCDiWS3E1z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,li_AzZnNZULMe4v73Smk,per_unit,line_item,Paquete HSM MEX,addon,paquete-hsm-3,NaN,tax_not_configured,li_Azz6ntUMU3km65lmw,txn_6olKGUM7GeKR72y7,...,Descuento de ajuste,percentage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2467,li_16BdPRULMXR8o76Yh,tiered,line_item,Agentes,addon,CLIENGO_STARTER_USERS,NaN,tax_not_configured,li_16BdHqUMTzMaePV0k,txn_16CLogUMfijSJ5fJ,...,Descuento de ajuste,percentage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2468,li_6ol3xULMdysQ59qN,tiered,line_item,Agentes - Prorated Charges,addon,CLIENGO_PREMIUM_CONVER_USERS,NaN,tax_not_configured,li_AzyjzNUMTzXq76PrJ,txn_AzyjxeUM7F1RA1kXD,...,Descuento de ajuste,percentage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2460,li_16BdPRULMQXnv6u2P,stairstep,line_item,Cliengo Starter,plan,CLIENGO_STARTER_50_V2,NaN,tax_not_configured,li_16BdHqUMTgCF5Ots6,txn_6olKGUM7Am8T6qRk,...,Descuento de ajuste,percentage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [292]:
inv.select_dtypes(include=['object'])

,sequence_prefix,activity_ids,activity_state,activity_user_id,activity_type_id,activity_type_icon,activity_date_deadline,activity_summary,access_url,access_warning,...,l10n_ar_afip_verification_type,l10n_ar_withholding_ids,l10n_cl_reference_ids,sale_type_id,sale_order_ids,picking_ids,website_id,l10n_ar_boarding_permission_ids,timesheet_ids,timesheet_encode_uom_id
0,,[],False,False,False,False,False,False,/my/invoices/35802,,...,not_available,[],[],False,[],[],False,[],[],"[4, Hours]"
1,,[],False,False,False,False,False,False,/my/invoices/35801,,...,not_available,[],[],False,[],[],False,[],[],"[4, Hours]"
2,,[],False,False,False,False,False,False,/my/invoices/35800,,...,not_available,[],[],False,[],[],False,[],[],"[4, Hours]"
3,,[],False,False,False,False,False,False,/my/invoices/35799,,...,not_available,[],[],False,[],[],False,[],[],"[4, Hours]"
4,,[],False,False,False,False,False,False,/my/invoices/35798,,...,not_available,[],[],False,[],[],False,[],[],"[4, Hours]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2307,BNK1/2024/,[],False,False,False,False,False,False,#,,...,not_available,[],[],False,[],[],False,[],[],"[4, Hours]"
2308,BNK1/2024/,[],False,False,False,False,False,False,#,,...,not_available,[],[],False,[],[],False,[],[],"[4, Hours]"
2309,BNK1/2024/,[],False,False,False,False,False,False,#,,...,not_available,[],[],False,[],[],False,[],[],"[4, Hours]"
2310,RBILL/2023/05/,[],False,False,False,False,False,False,/my/invoices/11,,...,not_available,[],[],False,[],[],False,[],[],"[4, Hours]"
